For my capstone, I downloaded my data from the EPA's NATIONAL WATER QUALITY MONITORING COUNCIL webpage and cleaned it here in this notebook. I also downloaded data for Cherry Creek Reservoir from another source but will not clean that until(if) I have created an accurate model with the EPA data. I cleaned the EPA data to to 'match' the Cherry Creak data in terms of features, specificlly water quality parameters. Therefore, many features and paramerters were removed from the EPA dataset. Also, as I am concerned with estimating Chlorophyll-a concentrations, I deleted all data collected at depths greater than the Cherry Creek photic zone of 3 m. In addition, the EPA data was very messy and I had to make many assumptions and heavily edit. 

# Data Wrangling

In [ ]:
import os
import pandas as pd
import string
import numpy as np
import csv
import re

I downloaded my data using the 'Advanced' tab and selected these options: https://www.waterqualitydata.us/#countrycode=US&siteType=Lake%2C%20Reservoir%2C%20Impoundment&sampleMedia=Water&sampleMedia=water&characteristicType=Inorganics%2C%20Major%2C%20Metals&characteristicType=Inorganics%2C%20Major%2C%20Non-metals&characteristicType=Inorganics%2C%20Minor%2C%20Metals&characteristicType=Inorganics%2C%20Minor%2C%20Non-metals&characteristicType=Nutrient&characteristicType=Physical&mimeType=csv&sorted=no&providers=NWIS&providers=STEWARDS&providers=STORET. Data had to be broken up into nine seperate CSV files for download to be successful.

In [ ]:
# Checked all csv data files to make sure they have the same column names in the same order before combining.

# Defined function
def check_csv_columns_in_folder(folder_path):
    # Got all CSV files in the specified folder and sorted them alphabetically
    csv_files = sorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])

    # Dictionary to store column names for each file
    columns_dict = {}

    # Iterated through each file
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as csv_file:
            # Read the first row (header) of the CSV file
            reader = csv.reader(csv_file)
            columns = next(reader, None)

            # Added column names to the dictionary
            columns_dict[file_path] = columns

    # Checked if all files have the same columns in the same order
    first_file_path = os.path.join(folder_path, csv_files[0])
    for file_path, columns in columns_dict.items():
        if columns != columns_dict[first_file_path]:
            print(f"Columns in {file_path} do not match the order of columns in {first_file_path}.")
            return False

    print("All CSV files in the folder have the same columns in the same order.")

    # Printed the list of files assessed in alphabetical order
    print("Files assessed:")
    for file_name in csv_files:
        print(file_name)

    return True

# Folder path
folder_path = "/Users/carahcampini/Desktop/Capstone/Raw_Data/"
check_csv_columns_in_folder(folder_path)

In [ ]:
# Combined CSV files into one file.

# Defined function to specify the folder path containing the EPA lake data CSV files
def stack_csv_files(folder_path, output_folder='/Users/carahcampini/Desktop/Capstone/Edited_Data', output_filename='EPA_data_01.csv'):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Initialized a list to store the filenames
    concatenated_files = []

    # Read the first CSV file to get the header
    first_file_path = os.path.join(folder_path, files[0])
    first_df = pd.read_csv(first_file_path)

    # Initialized the concatenated data frame with the first file
    concatenated_df = first_df
    concatenated_files.append(first_file_path)

    # Concatenated the remaining CSV files
    for file in files[1:]:
        file_path = os.path.join(folder_path, file)
        print(f"Processing file: {file_path}")
        df = pd.read_csv(file_path)
        concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)
        concatenated_files.append(file_path)

    # Wrote the concatenated data frame to a new CSV file in the specified output folder
    output_path = os.path.join(output_folder, output_filename)
    concatenated_df.to_csv(output_path, index=False)

    # Confirmed the file was created
    print(f"EPA CSV files in the EPA_data folder combined into '{output_path}'.")
    
    # Printed the list of concatenated files
    print("Concatenated files:")
    for file in concatenated_files:
        print(file)

# Called the function to stack CSV files
stack_csv_files('/Users/carahcampini/Desktop/Capstone/Raw_Data/')

In [ ]:
# Read EPA_data_01 file into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_01.csv')

## Reduced size of EPA data by removing rows and features

In [ ]:
# Investigated shape of EPA_data.
EPA_df.shape

That is a lot of data. My computer doesn't have enough memory to work with all of that. Therefore, this section focusses of reducing the size of the CSV file by removing features and row.

In [ ]:
# Counted number of unique values in each feature.
for column in EPA_df.columns:
    num_unique_values = EPA_df[column].nunique()
    print(f"'{column}': {num_unique_values}")

### Removed empty features

In [ ]:
# Looked to see how many features contain no data.

# Identified features with zero unique values
zero_unique_features = []
for column in EPA_df.columns:
    unique_count = EPA_df[column].nunique()
    if unique_count == 0:
        zero_unique_features.append(column)

# Listed features with zero unique values along with their unique values
for feature in zero_unique_features:
    unique_values = EPA_df[feature].unique()
    print(f"'{feature}': {unique_values}")

All the unique values are NaN and not usefull to me.

In [ ]:
# Deleted features with zero unique values.
EPA_df.drop(columns=zero_unique_features, inplace=True)

### Removed rows

In [ ]:
# Printed unique values for each feature to see what the data looks like.
for column in EPA_df.columns:
    unique_values = EPA_df[column].unique()
    print(f"{column}:", unique_values)
    print()

In [ ]:
# Removed rows where 'ResultStatusIdentifier' is 'Rejected' becasue this data is not usable.
EPA_df.drop(EPA_df[EPA_df['ResultStatusIdentifier'] == 'Rejected'].index, inplace=True)

In [ ]:
# Removed rows where 'ResultSampleFractionText' is 'Bed Sediment' becasue this data if from greater than 3m.
EPA_df.drop(EPA_df[EPA_df['ResultSampleFractionText'] == 'Bed Sediment'].index, inplace=True)

In [ ]:
# Removed rows where 'SubjectTaxonomicName' is anything but NaN becasue this data is not relevant. I am not concerned with taxonomic data.
EPA_df = EPA_df[EPA_df['SubjectTaxonomicName'].isna()]

In [ ]:
# Removed rows where 'SampleTissueAnatomyName' is anything but NaN becasue this data is not relevant. I am not concerned with tissue data.
EPA_df = EPA_df[EPA_df['SampleTissueAnatomyName'].isna()]

In [ ]:
# Removed rows where 'ResultAnalyticalMethod/MethodQualifierTypeName' has values 'duplicate records' or 'duplicates' becasue no duplicates should be in the data.
values_to_remove = ['duplicate records', 'duplicates']
EPA_df.drop(EPA_df[EPA_df['ResultAnalyticalMethod/MethodQualifierTypeName'].isin(values_to_remove)].index, inplace=True)

In [ ]:
# Removed unneeded paremeters. These parameters are not in common with the Cherry Creek data.
values_to_remove = ['Secchi Reading Condition (choice list)', 'Specific Conductance, Calculated/Measured Ratio', 'Temperature, sample']
EPA_df.drop(EPA_df[EPA_df['CharacteristicName'].isin(values_to_remove)].index, inplace=True)

In [ ]:
# Removed rows where 'ResultLaboratoryCommentText' indicated a failed analysis.
values_to_remove = [
    'Failed. Quality control criteria exceeded during analysis.',
    'Failed. Spiked lab blank recovery not acceptable.',
    'Equipment failed, sample not analyzed',
    'Failed. Lab performance check not acceptable.',
    'Failed. Matrix spike recovery not acceptable.',
    'Lab Failed, sample not analyzed'
]
EPA_df = EPA_df[~EPA_df['ResultLaboratoryCommentText'].isin(values_to_remove)]

In [ ]:
# Removed duplicate rows.
EPA_df = EPA_df.drop_duplicates()

### Deleted unnecessary features

In [ ]:
# Printed head of specific columns to leanr more about them and determine if they can be removed from dataframe.
print(EPA_df['ResultAnalyticalMethod/MethodIdentifier'].head(10))
print(EPA_df['ResultAnalyticalMethod/MethodIdentifierContext'].head(10))
print(EPA_df['ResultAnalyticalMethod/MethodName'].head(10))
print(EPA_df['ResultAnalyticalMethod/MethodQualifierTypeName'].head(10))
print(EPA_df['MethodDescriptionText'].head(10))

They can be removed.

In [ ]:
# Dropped feature because they do not contain necessary information.
features_to_drop = [
    'OrganizationFormalName',
    'DataLoggerLine',
    'ActivityStartTime/TimeZoneCode',
    'ResultStatusIdentifier',
    'StatisticalBaseCode',
    'ResultValueTypeName',
    'ResultTimeBasisText',
    'ResultTemperatureBasisText',
    'ResultParticleSizeBasisText',
    'PrecisionValue',
    'DataQuality/BiasValue',
    'ResultCommentText',
    'ResultSamplingPointName',
    'BiologicalIntentName',
    'SubjectTaxonomicName',
    'SampleTissueAnatomyName',
    'ResultAnalyticalMethod/MethodQualifierTypeName',
    'LaboratoryName',
    'AnalysisStartDate',
    'AnalysisStartTime/Time',
    'AnalysisStartTime/TimeZoneCode',
    'AnalysisEndDate',
    'AnalysisEndTime/Time',
    'AnalysisEndTime/TimeZoneCode',
    'ResultLaboratoryCommentCode',
    'ResultDetectionQuantitationLimitUrl',
    'LaboratoryAccreditationIndicator',
    'LaboratoryAccreditationAuthorityName',
    'TaxonomistAccreditationIndicator']
EPA_df.drop(columns=features_to_drop, inplace=True)

### Examined sample depth column

In [ ]:
# Checked to see if 'ResultDepthHeightMeasure/MeasureValue' contain datatypes other than float and what those values are.

# Extracted unique values for each data type
unique_values_by_dtype = {}

for dtype in EPA_df['ResultDepthHeightMeasure/MeasureValue'].apply(lambda x: type(x)).unique():
    values = EPA_df[(EPA_df['ResultDepthHeightMeasure/MeasureValue'].apply(lambda x: type(x)) == dtype)]['ResultDepthHeightMeasure/MeasureValue']
    unique_values = values.unique()
    unique_values.sort()
    unique_values_by_dtype[dtype] = unique_values

# Printed the count and unique values for each data type
for dtype, unique_values in unique_values_by_dtype.items():
    print(f"{dtype.__name__}: {len(unique_values)}, {unique_values}")

Lots of non-numeric depths that need to be edited.

In [ ]:
# Printed unique values containing non-numerical values.
values_with_letters = EPA_df[
    EPA_df['ResultDepthHeightMeasure/MeasureValue'].apply(lambda x: any(c.isalpha() or (c in string.punctuation and c != '.') for c in str(x)))
]['ResultDepthHeightMeasure/MeasureValue']

unique_values_with_letters = values_with_letters.value_counts()
print(unique_values_with_letters)

#### Removed rows

In [ ]:
# Looked at row contianing '19:40' to try to determine what value should be.
row_containing_1940 = EPA_df[EPA_df['ResultDepthHeightMeasure/MeasureValue'] == '19:40']
row_containing_1940

Could not determine what 19:40 should be.

In [ ]:
# Removed rows where 'ResultStatusIdentifier' is a string that indicates sample was not taken in surface 3m and '19:40'.
EPA_df.drop(EPA_df[EPA_df['ResultDepthHeightMeasure/MeasureValue'].isin(['HYPO', 'BOTTOM', 'THERMOCLINE', '19:40'])].index, inplace=True)

In [ ]:
# Printed the values in the 'ResultDepthHeightMeasure/MeasureUnitCode' fot the 'ResultDepthHeightMeasure/MeasureValue' of other string values to determine what they shopuld be.

# Specified the values to look for
values_to_look_for = ['3.0 M', 'EPI', 'Surface']

# Filtered rows based on multiple values
filtered_rows = EPA_df[EPA_df['ResultDepthHeightMeasure/MeasureValue'].isin(values_to_look_for)]

# Printed the unique values
for value in values_to_look_for:
    subset = filtered_rows[filtered_rows['ResultDepthHeightMeasure/MeasureValue'] == value]
    print(f"Value: {value}, Corresponding MeasureUnitCode: {subset['ResultDepthHeightMeasure/MeasureUnitCode'].unique()}")

I will assume that 3.0 M is in meters, not ft.

#### Edited values

In [ ]:
# Replaced 'ResultDepthHeightMeasure/MeasureValue' taken in the surface 3m with a numberical value. Replacement values are according to 'ResultDepthHeightMeasure/MeasureUnitCode'. 3.0 M will be converted back to meters in later steps.
EPA_df['ResultDepthHeightMeasure/MeasureValue'].replace({'EPI': 0.0, 'Surface': 0.0, '3.0 M': 9.84252}, inplace=True)

In [ ]:
# Converted string values of the 'ResultDepthHeightMeasure/MeasureValue' column to floats.
EPA_df['ResultDepthHeightMeasure/MeasureValue'] = EPA_df['ResultDepthHeightMeasure/MeasureValue'].astype(float)

In [ ]:
# Converted negative values to 0 in 'ResultDepthHeightMeasure/MeasureValue' column becasue a depth can't be negative. It's possible that some of these number represent a distance from the surface set at 0. However, if this is the case, most of the negative depths are within 3 m of the surface anyways and would be retained in the dataset.
EPA_df['ResultDepthHeightMeasure/MeasureValue'] = EPA_df['ResultDepthHeightMeasure/MeasureValue'].apply(lambda x: max(0, x))

##### Converted units

In [ ]:
# Printed unique values in 'ResultDepthHeightMeasure/MeasureUnitCode' to see which depth units are present.
print(EPA_df['ResultDepthHeightMeasure/MeasureUnitCode'].unique())

In [ ]:
# Converted 'ResultDepthHeightMeasure/MeasureValue' to meters and changed 'ResultDepthHeightMeasure/MeasureUnitCode' to m.
# Conversion factors
conversion_factors = {'ft': 0.3048, 'in': 0.0254, 'cm': 0.01}
# Applied conversions based on 'ResultDepthHeightMeasure/MeasureUnitCode'
for unit_code, conversion_factor in conversion_factors.items():
    mask = EPA_df['ResultDepthHeightMeasure/MeasureUnitCode'] == unit_code
    EPA_df.loc[mask, 'ResultDepthHeightMeasure/MeasureValue'] *= conversion_factor
    EPA_df.loc[mask, 'ResultDepthHeightMeasure/MeasureUnitCode'] = 'm'

##### Deleted rows with depth greater than 3 m

In [ ]:
# Counted number of values in ResultDepthHeightMeasure/MeasureUnitCode' greater than 3 and less than or equal to 3 to see how many rows will be deleted.
counts = EPA_df['ResultDepthHeightMeasure/MeasureValue'].apply(lambda x: '>3' if x > 3 else '<=3').value_counts()
print(counts)

In [ ]:
# Droped all rows where 'ResultDepthHeightMeasure/MeasureValue' is greater than 3 meters depth or Nan. The model to be devoped with this dataset is only concerned with the photic zone (>3 M).
EPA_df.drop(EPA_df[EPA_df['ResultDepthHeightMeasure/MeasureValue'] > 3].index, inplace=True)

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data_02.csv', index=False)

## Examined columns that will be used in model

In [ ]:
# Read EPA_data_02.csv into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_02.csv')

In [ ]:
# Made list of remaining columns
column_names = EPA_df.columns.tolist()
for name in column_names:
    print(name)

### Examined date and time columns

In [ ]:
# Printed count of dates and listed unique values.

# Printed total 'ActivityStartDate' counts.
data_type_counts = EPA_df['ActivityStartDate'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total Date counts and unique values
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ActivityStartDate'][EPA_df['ActivityStartDate'].apply(lambda x: type(x).__name__) == data_type].unique()
    unique_count = len(unique_values)
    
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    print(f"Unique Values: {unique_values}")

In [ ]:
# Printed count of start times and listed unique values.

# Printed total 'activityStartTime/Time' counts.
data_type_counts = EPA_df['ActivityStartTime/Time'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total Time counts and unique values
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ActivityStartTime/Time'][EPA_df['ActivityStartTime/Time'].apply(lambda x: type(x).__name__) == data_type].unique()
    unique_count = len(unique_values)
    
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    print(f"Unique Values: {unique_values}")

Date and time should be combined into one feature and correctly formatted.

In [ ]:
# Created 'DateTime' by combining 'ActivityStartDate' and 'ActivityStartTime/Time'.
EPA_df['DateTime'] = EPA_df.apply(lambda row: row['ActivityStartDate'] + " " + row['ActivityStartTime/Time'] if not pd.isna(row['ActivityStartTime/Time']) else row['ActivityStartDate'], axis=1)

In [ ]:
# Changed 'DateTime' from string to timestamp and formatted.
EPA_df['DateTime'] = pd.to_datetime(EPA_df['DateTime'], errors='ignore', format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Printed count of DateTime and listed unique values.

# Printed total 'DateTime' counts.
data_type_counts = EPA_df['DateTime'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total 'DateTime' and unique values
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['DateTime'][EPA_df['DateTime'].apply(lambda x: type(x).__name__) == data_type].unique()
    unique_count = len(unique_values)
    
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    print(f"Unique Values: {unique_values}")

In [ ]:
# Dropped Date and Time columns because they are no longer needed.
EPA_df.drop(columns=['ActivityStartDate', 'ActivityStartTime/Time'], inplace=True)

In [ ]:
# Saved to CSV.
EPA_df.to_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_03.csv', index=False)

### Examined water quality parameter columns

In [ ]:
# Read EPA_data_03.csv into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_03.csv')

In [ ]:
# Duplicated and combined features related to parameters so I have information in one place for future reference.
EPA_df['ParameterOrig'] = (
    EPA_df['CharacteristicName'].fillna('') +
    ' ' +
    EPA_df['ResultSampleFractionText'].fillna('') +
    ' ' +
    EPA_df['MethodSpecificationName'].fillna('') +
    '' +
    EPA_df['ResultMeasure/MeasureUnitCode'].fillna('')
)

In [ ]:
# Printed count of 'MethodSpecificationName' and listed unique values.

# Printed total 'MethodSpecificationName' counts
data_type_counts = EPA_df['MethodSpecificationName'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total 'MethodSpecificationName' counts and unique values
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['MethodSpecificationName'][EPA_df['MethodSpecificationName'].apply(lambda x: type(x).__name__) == data_type].unique()
    unique_count = len(unique_values)
    
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    print(f"Unique Values: {unique_values}")

In [ ]:
# Replaced 'unknown' with 'NaN' and 'as Chlorophyll' with 'as Chlorophyll a' for consistancey in naming.
EPA_df['MethodSpecificationName'].replace({'unknown': np.nan, 'as Chlorophyll': 'as Chlorophyll a'}, inplace=True)

In [ ]:
# Printed count of 'CharacteristicName' and listed unique values.

# Printed total 'CharacteristicName' counts
data_type_counts = EPA_df['CharacteristicName'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total 'CharacteristicName' counts and unique values
for data_type, count in data_type_counts.items():
    unique_values = sorted(EPA_df['CharacteristicName'][EPA_df['CharacteristicName'].apply(lambda x: type(x).__name__) == data_type].unique())
    unique_count = len(unique_values)
    
    print(f"\nData Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    for value in unique_values:
        print(value)

In [ ]:
# Changed 'CharacteristicName' values to make consistant.

# Mapping for replacements
replace_mapping = {
    'Specific conductivity***retired***use Specific conductance': 'Specific conductance',
    'Inorganic nitrogen (ammonia, nitrate and nitrite)***retired***use Inorganic nitrogen (NO2, NO3, & NH3)': 'Inorganic nitrogen (NO2, NO3, & NH3)',
    'Inorganic nitrogen (nitrate and nitrite) ***retired***use Nitrate + Nitrite': 'Inorganic nitrogen (NO2 & NO3)',
    'Inorganic nitrogen (nitrate and nitrite)': 'Inorganic nitrogen (NO2 & NO3)',
    'Inorganic nitrogen (nitrate and nitrite) as N': 'Inorganic nitrogen (NO2 & NO3) as N',
    'Nitrate + Nitrite': 'Inorganic nitrogen (NO2 & NO3)',
    'Chlorophyll': 'Chlorophyll a',
    'Chlorophyll a (probe relative fluorescence)': 'Chlorophyll a',
    'Chlorophyll a (probe)': 'Chlorophyll a',
    'Chlorophyll a - Phytoplankton (suspended)': 'Chlorophyll a',
    'Chlorophyll a, uncorrected for pheophytin': 'Chlorophyll a',
}

# Replaced values in 'CharacteristicName' using the mapping for full matches
EPA_df['CharacteristicName'] = EPA_df['CharacteristicName'].replace(replace_mapping)

In [ ]:
# Printed count of 'ResultSampleFractionText' and listed unique values.


# Printed total 'ResultSampleFractionText' counts
data_type_counts = EPA_df['ResultSampleFractionText'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total 'ResultSampleFractionText' counts and unique values
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ResultSampleFractionText'][EPA_df['ResultSampleFractionText'].apply(lambda x: type(x).__name__) == data_type].unique()
    unique_values = sorted(unique_values)  # Sort the unique values alphabetically
    unique_count = len(unique_values)
    
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    print(f"Unique Values:")
    for value in unique_values:
        print(value)

In [ ]:
# Changed some 'ResultSampleFractionText' values to make consistant.

# Mapping for replacements
replace_mapping = {
    ' ': np.NaN,
    'Non-filterable': 'Total',
    'Non-Filterable (Particle)': 'Total',
    'Unfiltered': 'Total',
    'Unfiltered, field': 'Total',
    'Filterable': 'Dissolved',
    'Filtered, field': 'Dissolved',
    'Filtered, lab': 'Dissolved',
    'Recoverable': 'Total',
    'Total Recoverable': 'Total',
    'Acid Soluble': np.NaN,
    'Field': np.NaN,
    'Fixed': np.NaN,
    'Free Available': 'Dissolved',
    'Supernate': 'Dissolved',
    'Suspended': np.NaN,
    'Total Residual': np.NaN,
    'Total Soluble': 'Dissolved'
}

# Replaced values in 'CharacteristicName' using the mapping for full matches
EPA_df['ResultSampleFractionText'] = EPA_df['ResultSampleFractionText'].replace(replace_mapping)

In [ ]:
# Combined the parameter columns so all info was in one column for more name editing.
EPA_df['Parameter'] = EPA_df[['CharacteristicName', 'ResultSampleFractionText', 'MethodSpecificationName']].fillna('').apply(lambda row: ', '.join(filter(None, row)), axis=1)

In [ ]:
# Cleaned up spaces in parameter values.
EPA_df['Parameter'] = EPA_df['Parameter'].str.replace('\s+', ' ').str.strip()

In [ ]:
# Printed unique values in 'Parameter' to see what needs to be edited for consistancy.
unique_values_parameter = sorted(EPA_df['Parameter'].unique())
for value in unique_values_parameter:
    print(value)

Names are inconsistant resulting in multiple versions for the same parameter, format is not consistant between perameters, and some information in a name contradictions other information. 

In [ ]:
# Changed 'MethodSpecificationName' portion of 'Parameter' values to fix data entry mistakes.

# Mapping for replacements
replace_mapping = {
    'Alkalinity, total, Total, as N': 'Alkalinity, Total',
    'Alkalinity, total, Total, as NO3': 'Alkalinity, Total',
    'Ammonia-nitrogen, Total, as CaCO3': 'Ammonia-nitrogen, Total',
    'Calcium hydroxide, as CaCO3': 'Calcium hydroxide',
    'Calcium hydroxide, Total, as CaCO3': 'Calcium hydroxide, Total',
    'Chloride, Dissolved, as P': 'Chloride, Dissolved',
    'Chloride, Total, as N': 'Chloride, Total',
    'Chlorophyll a, Total, as N': 'Chlorophyll a',
    'Chlorophyll a, Total, as S': 'Chlorophyll a',
    'Chlorophyll a, as CaCO3': 'Chlorophyll a',
    'Chlorophyll a, as Cl': 'Chlorophyll a',
    'Chlorophyll a, as Cl': 'Chlorophyll a',
    'Chlorophyll a, as N': 'Chlorophyll a',
    'Chlorophyll a, as P': 'Chlorophyll a',
    'Chlorophyll a, as PO4': 'Chlorophyll a',
    'Chlorophyll a, as SO4': 'Chlorophyll a',
    'Chlorophyll a, as SiO2': 'Chlorophyll a',
    'Conductivity, as N': 'Conductivity',
    'Conductivity, as PO4': 'Conductivity',
    'Dissolved oxygen (DO), as N': 'Dissolved oxygen (DO)',
    'Dissolved oxygen (DO), as O2': 'Dissolved oxygen (DO)',
    'Dissolved oxygen (DO), as PO4': 'Dissolved oxygen (DO)',
    'Hardness, carbonate, Total, as P': 'Hardness, carbonate, Total',
    'Hardness, non-carbonate, Dissolved, as CaCO3': 'Hardness, non-carbonate, Dissolved',
    'Hardness, non-carbonate, Total, as CaCO3': 'Hardness, non-carbonate, Total',
    'Hydroxide, Total, as CaCO3': 'Hydroxide, Total',
    'Kjeldahl nitrogen, Total, as CaCO3': 'Kjeldahl nitrogen, Total',
    'Kjeldahl nitrogen, Total, as NH4': 'Kjeldahl nitrogen, Total',
    'Kjeldahl nitrogen, Total, as NO3': 'Kjeldahl nitrogen, Total',
    'Nitrate, Dissolved, as NO2': 'Nitrate, Dissolved',
    'Nitrate, Total, as NO2': 'Nitrate, Total',
    'Nitrogen, mixed forms (NH3), (NH4), organic, (NO2) and (NO3), Total, as NO3': 'Nitrogen, mixed forms (NH3), (NH4), organic, (NO2) and (NO3), Total',
    'Nitrogen, Dissolved, as NO2': 'Nitrogen, Dissolved',
    'Nitrogen, Dissolved, as NO3': 'Nitrogen, Dissolved',
    'Phosphorus, Dissolved, as N': 'Phosphorus, Dissolved',
    'Phosphorus, Total, as N': 'Phosphorus, Total',
    'Sulfate, Total, as CaCO3': 'Sulfate, Total',
    'Sulfate, Total, as Cl': 'Sulfate, Total',
    'Sulfate, Total, as N': 'Sulfate, Total',
    'Sulfate, Total, as P': 'Sulfate, Total',
    'Sulfate, Total, as S': 'Sulfate, Total',
    'Temperature, water, as N': 'Temperature, water',
    'Temperature, water, as PO4': 'Temperature, water',
    'pH, Dissolved, as pH': 'pH, Dissolved',
    'pH, as N': 'pH',
    'pH, as PO4': 'pH',
    'pH, as pH': 'pH'
}

# Replaced values in 'CharacteristicName' using the mapping for full matches
EPA_df['Parameter'] = EPA_df['Parameter'].replace(replace_mapping)

In [ ]:
# Printed list of unique 'ResultMeasure/MeasureUnitCode' to see what could be helpful for naming parameters.

# Printed total 'ResultMeasure/MeasureUnitCode' counts
data_type_counts = EPA_df['ResultMeasure/MeasureUnitCode'].apply(lambda x: type(x).__name__).value_counts()

# Printed the total 'ResultMeasure/MeasureUnitCode' counts and unique values
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ResultMeasure/MeasureUnitCode'][EPA_df['ResultMeasure/MeasureUnitCode'].apply(lambda x: type(x).__name__) == data_type].unique()
    unique_values = sorted(unique_values)  # Sort the unique values alphabetically
    unique_count = len(unique_values)
    
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_count}")
    print(f"Unique Values:")
    for value in unique_values:
        print(value)

In [ ]:
# Selected 'ResultMeasure/MeasureUnitCode' values that provide more information for 'Paramters'.

# Filtered for 'ResultMeasure/MeasureUnitCode'
filtered_df = EPA_df[EPA_df['ResultMeasure/MeasureUnitCode'].isin(['mg/l CaCO3**', 'mg/l CaCO3', 'mg/l asNO3', 'mg/l as N', 'mg/l asPO4', 'mg/l as P', 'mg/l NH4', 'mg/l NO3', 'mg/l asNO2', 'mg/l PO4', 'mg/kg as N', 'mg/kg as P', 'ug/L as P', 'ug/l as P'])]

# Displayed filtered 'ResultMeasure/MeasureUnitCode' and 'Paramter' values
unique_pairs = filtered_df[['Parameter', 'ResultMeasure/MeasureUnitCode']].drop_duplicates().sort_values(by='Parameter')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
unique_pairs

In [ ]:
# Printed full name so I could see full length.
value_at_index = EPA_df.at[23521161, 'Parameter']
print(value_at_index)

In [ ]:
# Printed full name so I could see full length.
value_at_index = EPA_df.at[14721778, 'Parameter']
print(value_at_index)

In [ ]:
# Printed full name so I could see full length.
value_at_index = EPA_df.at[3771489, 'Parameter']
print(value_at_index)

In [ ]:
# Added info from 'ResultMeasure/MeasureUnitCode' to 'Parameter' where above table could not distinguish.

# Defined the conditions for updating the 'Parameter' column
conditions = [
    (EPA_df['Parameter'] == "Nitrate, Dissolved") | 
    (EPA_df['Parameter'] == "Nitrate, Total") | 
    (EPA_df['Parameter'] == "Nitrite, Dissolved") | 
    (EPA_df['Parameter'] == "Nitrite, Total") | 
    (EPA_df['Parameter'] == "Nitrogen, mixed forms (NH3), (NH4), organic") | 
    (EPA_df['Parameter'] == "Orthophosphate, Dissolved") | 
    (EPA_df['Parameter'] == "Phosphorus, Dissolved") | 
    (EPA_df['Parameter'] == "Phosphorus, Total") | 
    (EPA_df['Parameter'] == "Alkalinity, Phenolphthalein (total hydroxide+1/2 carbonate), Total") | 
    (EPA_df['Parameter'] == "Nitrogen, mixed forms (NH3), (NH4), organic, (NO2) and (NO3), Dissolved") | 
    (EPA_df['Parameter'] == "Nitrogen, mixed forms (NH3), (NH4), organic, (NO2) and (NO3), Total"),
    EPA_df['ResultMeasure/MeasureUnitCode'].notna()
]

# Defined the values to be added from 'ResultMeasure/MeasureUnitCode' column
values_to_add = EPA_df['ResultMeasure/MeasureUnitCode']

# Updated the 'Parameter' column based on the conditions
EPA_df['Parameter'] = np.where(conditions[0] & conditions[1], EPA_df['Parameter'] + ', ' + values_to_add, EPA_df['Parameter'])

In [ ]:
# Printed unique values in 'Parameter' to see what needs to be edited to make consistant.
unique_values_parameter = sorted(EPA_df['Parameter'].unique())
for value in unique_values_parameter:
    print(value)

In [ ]:
# Edited 'Parameters' to fix entry errors and remove unnecessary information.

# Defined a mapping for replacements
replacement_mapping = {
    'total': 'Total',
    'bicarbonate': 'Bicarbonate',
    'ammonium': 'Ammonium',
    r'\bcarbonate\b': 'Carbonate',
    'nitrogen': 'Nitrogen',
    'oxygen': 'Oxygen',
    'saturation': 'Saturation',
    'non-': 'Non-',
    'phosphorus': 'Phosphorus',
    'transmissivity': 'Transmissivity',
    'attenuation': 'Attenuation',
    'depth': 'Depth',
    'asNO3': 'as NO3',
    'asNO2': 'as NO2',
    'asPO4': 'as PO4',
    'mixed forms': 'Mixed Forms',
    'organic': 'Organic',
    'transparency': 'Transparency',
    r'\bion\b': 'Ion',
    'carbon': 'Carbon',
    'solids': 'Solids',
    'suspended': 'Suspended',
    'volatile': 'Volatile',
    'hydroxide': 'Hydroxide',
    ' mg/l': '',
    ' mg/L': '',
    ' ug/L': '',
    '\*\*': '',
    ' ug/l': '',
    ' %': '',
    ' mg/kg': '',
    'ppb': '',
    'ppm': '',
    ', sample': '',
    ', water': '',
    ' disc': ''
}

# Applied replacements to the 'Parameter' column for partial matches
EPA_df['Parameter'] = EPA_df['Parameter'].replace(replacement_mapping, regex=True)

In [ ]:
# Removed commas and spaces at the end of values in the 'Parameter' column.
EPA_df['Parameter'] = EPA_df['Parameter'].str.rstrip(',')

In [ ]:
# Printed unique values in 'Parameter' to see what needs to be edited to make consistant.
unique_values_parameter = sorted(EPA_df['Parameter'].unique())
for value in unique_values_parameter:
    print(value)

In [ ]:
# Saved to CSV
EPA_df.to_csv('EPA_data_04.csv', index=False)

In [ ]:
# Read EPA_data_04 file into pandas DataFrame
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_04.csv')

In [ ]:
# Changed 'Parameter' values to make consistant.

# Mapping for replacements
replace_mapping = {
    'Alkalinity': 'Alkalinity, Total, as CaCO3',
    'Alkalinity, Total': 'Alkalinity, Total, as CaCO3',
    'Alkalinity, Total, Total': 'Alkalinity, Total, as CaCO3',
    'Alkalinity, Total, Total, as CaCO3': 'Alkalinity, Total, as CaCO3',
    'Alkalinity, Total, as CaCO3': 'Alkalinity, Total, as CaCO3',
    'Alkalinity, Total, InOrganic': 'Alkalinity, Total, as CaCO3',
    'Alkalinity, Dissolved': 'Alkalinity, Dissolved, as CaCO3',
    'Alkalinity, Total, Dissolved': 'Alkalinity, Dissolved, as CaCO3',
    'Alkalinity, Total, Dissolved, as CaCO3': 'Alkalinity, Dissolved, as CaCO3',
    'Alkalinity, BiCarbonate': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'BiCarbonate': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'Alkalinity, BiCarbonate as CaCO3': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'Alkalinity, BiCarbonate, as CaCO3': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'BiCarbonate, as CaCO3': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'Alkalinity, BiCarbonate, Total': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'Alkalinity, BiCarbonate, Total, as CaCO3': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'Alkalinity, BiCarbonate as CaCO3, Total': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'BiCarbonate, Total': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'BiCarbonate, Total, as CaCO3': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'BiCarbonate, InOrganic': 'Alkalinity, Bicarbonate, Total, as CaCO3',
    'Alkalinity, BiCarbonate, Dissolved': 'Alkalinity, Bicarbonate, Dissolved, as CaCO3',
    'Alkalinity, BiCarbonate, Dissolved, as CaCO3': 'Alkalinity, Bicarbonate, Dissolved, as CaCO3',
    'BiCarbonate, Dissolved': 'Alkalinity, Bicarbonate, Dissolved, as CaCO3',
    'BiCarbonate, Dissolved, as CaCO3': 'Alkalinity, Bicarbonate, Dissolved, as CaCO3',
    'BiCarbonate, as HCO3': 'Alkalinity, Bicarbonate, Total, as HCO3',
    'BiCarbonate, Dissolved, as HCO3': 'Alkalinity, Bicarbonate, Dissolved, as HCO3',
    'BiCarbonate, Total, as HCO3': 'Alkalinity, Bicarbonate, Total, as HCO3',
    'Alkalinity, Carbonate': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Carbonate': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Alkalinity, Carbonate, Total': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Carbonate, Total': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Alkalinity, Carbonate as CaCO3, Total': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Carbonate, Total, as CaCO3': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Carbonate, as CaCO3': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Alkalinity, Carbonate as CaCO3': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Alkalinity, Carbonate, as CaCO3': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Carbonate, InOrganic': 'Alkalinity, Carbonate, Total, as CaCO3',
    'Alkalinity, Carbonate, Dissolved': 'Alkalinity, Carbonate, Dissolved, as CaCO3',
    'Carbonate, Dissolved': 'Alkalinity, Carbonate, Dissolved, as CaCO3',
    'Carbonate, Dissolved, as CaCO3': 'Alkalinity, Carbonate, Dissolved, as CaCO3',
    'Carbonate, Total, as CO3': 'Alkalinity, Carbonate, Total, as CO3',
    'Carbonate, Dissolved, as CO3': 'Alkalinity, Carbonate, Dissolved, as CO3',
    'Alkalinity, Hydroxide': 'Alkalinity, Hydroxide, Total, as CaCO3',
    'Alkalinity, Hydroxide as CaCO3': 'Alkalinity, Hydroxide, Total, as CaCO3',
    'Alkalinity, Hydroxide, as CaCO3': 'Alkalinity, Hydroxide, Total, as CaCO3',
    'Hydroxide, Total': 'Alkalinity, Hydroxide, Total, as CaCO3',
    'Alkalinity, Hydroxide, Total': 'Alkalinity, Hydroxide, Total, as CaCO3',
    'Alkalinity, Hydroxide as CaCO3, Total': 'Alkalinity, Hydroxide, Total, as CaCO3',
    'Alkalinity, Hydroxide, Dissolved': 'Alkalinity, Hydroxide, Dissolved, as CaCO3',
    'Hydroxide, Dissolved': 'Alkalinity, Hydroxide, Dissolved, as CaCO3',
    'Hydroxide, Total, as OH': 'Alkalinity, Hydroxide, Total, as OH',
    'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate)': 'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Total, as CaCO3',
    'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Total': 'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Total, as CaCO3',
    'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Total, CaCO3': 'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Total, as CaCO3',
    'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), as CaCO3': 'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Total, as CaCO3',
    'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Dissolved': 'Alkalinity, Phenolphthalein (Total Hydroxide+1/2 Carbonate), Dissolved, as CaCO3',
    'Bromide': 'Bromide, Total',
    'Calcium Carbonate': 'Calcium Carbonate, Total, as CaCO3',
    'Calcium Carbonate, Total': 'Calcium Carbonate, Total, as CaCO3',
    'Calcium Carbonate, as CaCO3': 'Calcium Carbonate, Total, as CaCO3',
    'Calcium Hydroxide': 'Calcium Hydroxide, Total, as CaCO3',
    'Carbon': 'Carbon, Total',
    'Total Carbon': 'Carbon, Total',
    'Total Carbon, Total': 'Carbon, Total',
    'Total Carbon, Total, as C': 'Carbon, Total',
    'Chloride': 'Chloride, Total',
    'Chloride, Total, as Cl': 'Chloride, Total',
    'Chloride, as Cl': 'Chloride, Total',
    'Chloride, InOrganic': 'Chloride, Total',
    'Chloride, Dissolved': 'Chloride, Dissolved',
    'Chloride, Dissolved, as Cl': 'Chloride, Dissolved',
    'Chloride, Total, as CaCO3': 'Chloride, Total',
    'Chlorophyll a': 'Chlorophyll a, Total',
    'Chlorophyll a, Total, as Chlorophyll a': 'Chlorophyll a, Total',
    'Chlorophyll a, as Chlorophyll a': 'Chlorophyll a, Total',
    'Conductivity': 'Conductivity, Total',
    'Specific conductance, Dissolved': 'Conductivity, Total',
    'Specific conductance, Total': 'Conductivity, Total',
    'Specific conductance': 'Conductivity, Total',
    'Dissolved Oxygen (DO), Dissolved': 'Dissolved Oxygen',
    'Dissolved Oxygen (DO), Total': 'Dissolved Oxygen',
    'Dissolved Oxygen (DO)': 'Dissolved Oxygen',
    'Dissolved Oxygen Saturation, Total': 'Dissolved Oxygen, Saturation',
    'Dissolved Oxygen Saturation, Dissolved': 'Dissolved Oxygen, Saturation',
    'Hardness, Carbonate': 'Hardness, Carbonate, Total, as CaCO3',
    'Hardness, Carbonate, Total': 'Hardness, Carbonate, Total, as CaCO3',
    'Hardness, Carbonate, as CaCO3': 'Hardness, Carbonate, Total, as CaCO3',
    'Hardness, Carbonate, Dissolved': 'Hardness, Carbonate, Dissolved, as CaCO3',
    'Hardness, Non-Carbonate': 'Hardness, Non-Carbonate, Total',
    'Light Attenuation, Depth at 99%, Total': 'Light Attenuation, Depth at 99%',
    'Nitrogen': 'Nitrogen, Total, as N',
    'Nitrogen Ion': 'Nitrogen, Total, as N',
    'Nitrogen Ion, Total': 'Nitrogen, Total, as N',
    'Nitrogen, Total': 'Nitrogen, Total, as N',
    'Nitrogen, as N': 'Nitrogen, Total, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3)': 'Nitrogen, Total, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), Total': 'Nitrogen, Total, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), Total, as N': 'Nitrogen, Total, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), as N': 'Nitrogen, Total, as N',
    'Total Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), Total, as N': 'Nitrogen, Total, as N',
    'Total Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), as N': 'Nitrogen, Total, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), Total, NO3': 'Nitrogen, Total, as N',
    'Nitrogen, Dissolved': 'Nitrogen, Dissolved, as N',
    'Nitrogen Ion, Dissolved': 'Nitrogen, Dissolved, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), Dissolved': 'Nitrogen, Dissolved, as N',
    'Nitrogen, Mixed Forms (NH3), (NH4), Organic, (NO2) and (NO3), Dissolved, as N': 'Nitrogen, Dissolved, as N',
    'InOrganic Nitrogen (NO2 & NO3)': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3)': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3) as N': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), Total, as N': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), as N': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), Total': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3) as N, Total': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), Total, as NO3': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), as NO3': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3) as N, Dissolved': 'Nitrogen (NO2 & NO3), Dissolved, as N',
    'InOrganic Nitrogen (NO2 & NO3), Dissolved': 'Nitrogen (NO2 & NO3), Dissolved, as N',
    'InOrganic Nitrogen (NO2 & NO3), Dissolved, as N': 'Nitrogen (NO2 & NO3), Dissolved, as N',
    'InOrganic Nitrogen (NO2 & NO3), Dissolved, as NO3': 'Nitrogen (NO2 & NO3), Dissolved, as N',
    'InOrganic Nitrogen (NO2, NO3, & NH3)': 'Nitrogen (NO2, NO3, & NH3), Total, as N',
    'InOrganic Nitrogen (NO2, NO3, & NH3), Total, as N': 'Nitrogen (NO2, NO3, & NH3), Total, as N',
    'InOrganic Nitrogen (NO2, NO3, & NH3), Total': 'Nitrogen (NO2, NO3, & NH3), Total, as N',
    'InOrganic Nitrogen (NO2, NO3, & NH3), Dissolved, as N': 'Nitrogen (NO2, NO3, & NH3), Dissolved, as N',
    'InOrganic Nitrogen (NO2 & NO3), InOrganic, as N': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), InOrganic, as NO3': 'Nitrogen (NO2 & NO3), Total, as N',
    'InOrganic Nitrogen (NO2 & NO3), Volatile, as N': 'Nitrogen (NO2 & NO3), Volatile, as N',
    'Nutrient-Nitrogen, Total': 'Nitrogen (NO3 & NH4), Total, as N',
    'Nutrient-Nitrogen, Dissolved': 'Nitrogen (NO3 & NH4), Dissolved, as N',
    'Nitrogen, InOrganic, as N': 'Nitrogen (NO2 & NO3), Total, as N',
    'Organic Nitrogen': 'Nitrogen, Total Organic, as N',
    'Organic Nitrogen, Total': 'Nitrogen, Total Organic, as N',
    'Organic Nitrogen, Total, as N': 'Nitrogen, Total Organic, as N',
    'Organic Nitrogen, Dissolved, as N': 'Nitrogen, Dissolved Organic, as N',
    'Organic Nitrogen, Dissolved': 'Nitrogen, Dissolved Organic, as N',
    'Kjeldahl Nitrogen': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Kjeldahl Nitrogen, Total': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Kjeldahl Nitrogen, Total, as N': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Kjeldahl Nitrogen, as N': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Total Kjeldahl Nitrogen, Total': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Total Kjeldahl Nitrogen, Total, as N': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Total Kjeldahl Nitrogen, as N': 'Nitrogen (NH4, org), Total Kjeldahl, as N',
    'Kjeldahl Nitrogen, Volatile, as N': 'Nitrogen (NH4, org), Total Volatile Kjeldahl, as N',
    'Kjeldahl Nitrogen, Dissolved': 'Nitrogen (NH4, org), Dissolved Kjeldahl, as N',
    'Kjeldahl Nitrogen, Dissolved, as N': 'Nitrogen (NH4, org), Dissolved Kjeldahl, as N',
    'Total Kjeldahl Nitrogen, Dissolved, as N': 'Nitrogen (NH4, org), Dissolved Kjeldahl, as N',
    'Ammonia': 'Ammonia (NH3), Total, as N',
    'Ammonia, Total, as N': 'Ammonia (NH3), Total, as N',
    'Ammonia, Total': 'Ammonia (NH3), Total, as N',
    'Ammonia and Ammonium, Total': 'Ammonia (NH3) and Ammonium (NH4), Total, as N',
    'Ammonia, as N': 'Ammonia (NH3), Total, as N',
    'Ammonia-Nitrogen as N': 'Ammonia (NH3), Total, as N',
    'Ammonia-Nitrogen, Total, as N': 'Ammonia (NH3), Total, as N',
    'Ammonia-Nitrogen as N, Total': 'Ammonia (NH3), Total, as N',
    'Ammonia-Nitrogen': 'Ammonia (NH3), Total, as N',
    'Ammonia-Nitrogen, Total': 'Ammonia (NH3), Total, as N',
    'Ammonia-nitrogen, Dissolved, as NH3': 'Ammonia, Dissolved, as NH3',
    'Ammonia-nitrogen, as NH3': 'Ammonia, Total, as NH3',
    'Ammonia-nitrogen, Total, as NH3': 'Ammonia, Total, as NH3',
    'Ammonia as NH3': 'Ammonia (NH3), Total, as NH3',
    'Ammonia, as NH3': 'Ammonia (NH3), Total, as NH3',
    'Ammonia as NH3, Total': 'Ammonia (NH3), Total, as NH3',
    'Ammonia, Total, as NH3': 'Ammonia (NH3), Total, as NH3',
    'Ammonia, Volatile, as N': 'Ammonia (NH3), Volatile, as N',
    'Ammonia, Dissolved': 'Ammonia (NH3), Dissolved, as N',
    'Ammonia, InOrganic': 'Ammonia (NH3), Total, as N',
    'Ammonia and Ammonium, Dissolved': 'Ammonia (NH3) and Ammonium (NH4), Dissolved, as N',
    'Ammonia-Nitrogen as N, Dissolved': 'Ammonia (NH3), Dissolved, as N',
    'Ammonia-Nitrogen, Dissolved': 'Ammonia (NH3), Dissolved, as N',
    'Ammonia-Nitrogen, Dissolved, as N': 'Ammonia (NH3), Dissolved, as N',
    'Ammonium, as N': 'Ammonium (NH4), Total, as N',
    'Ammonium as N, Total': 'Ammonium (NH4), Total, as N',
    'Ammonium, Total': 'Ammonium (NH4), Total, as N',
    'Ammonium as NH4, Total': 'Ammonium (NH4), Total, as NH4',
    'Ammonium, Dissolved': 'Ammonium (NH4), Dissolved, as N',
    'Ammonia, Dissolved, as N': 'Ammonia (NH3), Dissolved, as N',
    'Ammonia, Dissolved, as NH3': 'Ammonia (NH3), Dissolved, as NH3',
    'Ammonia, Dissolved, as NH4': 'Ammonia (NH3), Dissolved, as NH3',
    'Ammonium, Dissolved, as N': 'Ammonium (NH4), Dissolved, as N',
    'Ammonium, Dissolved, as NH4': 'Ammonium (NH4), Dissolved, as NH4',
    'Ammonium, Total, as N': 'Ammonium (NH4), Total, as N',
    'Ammonium, Total, as NH4': 'Ammonium (NH4), Total, as NH4',
    'Nitrate': 'Nitrate (NO3), Total, as N',
    'Nitrate as N': 'Nitrate (NO3), Total, as N',
    'Nitrate as N, Total': 'Nitrate (NO3), Total, as N',
    'Nitrate, Total': 'Nitrate (NO3), Total, as N',
    'Nitrate, Total, as N': 'Nitrate (NO3), Total, as N',
    'Nitrate, as N': 'Nitrate (NO3), Total, as N',
    'Nitrate, InOrganic, as N': 'Nitrate (NO3), Total, as N',
    'Nitrate, Total, as NO3': 'Nitrate (NO3), Total, as NO3',
    'Nitrate, as NO3': 'Nitrate (NO3), Total, as NO3',
    'Nitrate as N, Dissolved': 'Nitrate (NO3), Dissolved, as N',
    'Nitrate, Dissolved': 'Nitrate (NO3), Dissolved, as N',
    'Nitrate, Dissolved, as N': 'Nitrate (NO3), Dissolved, as N',
    'Nitrate, Dissolved, as NO3': 'Nitrate (NO3), Dissolved, as NO3',
    'Nitrite': 'Nitrite (NO2), Total, as N',
    'Nitrite as N': 'Nitrite (NO2), Total, as N',
    'Nitrite as N, Total': 'Nitrite (NO2), Total, as N',
    'Nitrite, Total': 'Nitrite (NO2), Total, as N',
    'Nitrite, Total, as N': 'Nitrite (NO2), Total, as N',
    'Nitrite, as N': 'Nitrite (NO2), Total, as N',
    'Nitrite, Total, as NO2': 'Nitrite (NO2), Total, as NO2',
    'Nitrite, as NO2': 'Nitrite (NO2), Total, as NO2',
    'Nitrite, Volatile, as N': 'Nitrite (NO2), Volatile, as N',
    'Nitrite as N, Dissolved': 'Nitrite (NO2), Dissolved, as N',
    'Nitrite, Dissolved': 'Nitrite (NO2), Dissolved, as N',
    'Nitrite, Dissolved, as N': 'Nitrite (NO2), Dissolved, as N',
    'Nitrite, Dissolved, as NO2': 'Nitrite (NO2), Dissolved, as NO2',
    'Oxidation reduction potential (ORP)': 'Oxidation Reduction Potential (ORP), Total',
    'Oxidation reduction potential (ORP), Dissolved': 'Oxidation Reduction Potential (ORP), Dissolved',
    'Oxidation reduction potential (ORP), Total': 'Oxidation Reduction Potential (ORP), Total',
    'Phosphorus': 'Phosphorus, Total, as P',
    'Phosphorus, Total': 'Phosphorus, Total, as P',
    'Phosphorus, Total, ': 'Phosphorus, Total, as P',
    'Phosphorus, as P': 'Phosphorus, Total, as P',
    'Phosphorus as P, Total': 'Phosphorus, Total, as P',
    'Phosphorus, Total, as P': 'Phosphorus, Total, as P',
    'Phosphorus, Total, PO4': 'Phosphorus, Total, as P',
    'Phosphorus, Total, as PO4': 'Phosphorus, Total, as P',
    'Phosphorus as P, Dissolved': 'Phosphorus, Dissolved, as P',
    'Phosphorus, Dissolved': 'Phosphorus, Dissolved, as P',
    'Phosphorus, Dissolved, as P': 'Phosphorus, Dissolved, as P',
    'Phosphorus, Dissolved, PO4': 'Phosphorus, Dissolved, as P',
    'Phosphorus, Dissolved, as PO4': 'Phosphorus, Dissolved, as P',
    'InOrganic Phosphorus': 'Phosphorus, Total Inorganic, as P',
    'InOrganic Phosphorus, Total': 'Phosphorus, Total Inorganic, as P',
    'Phosphorus, InOrganic, as P': 'Phosphorus, Total Inorganic, as P',
    'Phosphorus, Total': 'Phosphorus, Total, as P',
    'Organic Phosphorus': 'Phosphorus, Total Organic, as P',
    'Organic Phosphorus, Total': 'Phosphorus, Total Organic, as P',
    'Organic Phosphorus, Total, as P': 'Phosphorus, Total Organic, as P',
    'Phosphorus, Particulate Organic, as P': 'Phosphorus, Total Particulate Organic, as P',
    'Organic Phosphorus, as P': 'Phosphorus, Total Organic, as P',
    'Organic Phosphorus, Dissolved': 'Phosphorus, Dissolved Organic, as P',
    'Organic Phosphorus, Dissolved, as P': 'Phosphorus, Dissolved Organic, as P',
    'Orthophosphate': 'Orthophosphate, Total, as P',
    'Orthophosphate as P': 'Orthophosphate, Total, as P',
    'Orthophosphate, as P': 'Orthophosphate, Total, as P',
    'Orthophosphate as P, Total': 'Orthophosphate, Total, as P',
    'Orthophosphate, Total': 'Orthophosphate, Total, as P',
    'Soluble Reactive Phosphorus (SRP)': 'Orthophosphate, Total, as P',
    'Soluble Reactive Phosphorus (SRP), Total': 'Orthophosphate, Total, as P',
    'Soluble Reactive Phosphorus (SRP), Total, as P': 'Orthophosphate, Total, as P',
    'Orthophosphate as PO4, Total': 'Orthophosphate, Total, as PO4',
    'Orthophosphate, Total, as PO4': 'Orthophosphate, Total, as PO4',
    'Orthophosphate, as PO4': 'Orthophosphate, Total, as PO4',
    'Orthophosphate as PO4': 'Orthophosphate, Total, as PO4',
    'Orthophosphate as P, Dissolved': 'Orthophosphate, Dissolved, as P',
    'Orthophosphate, Dissolved, as P': 'Orthophosphate, Dissolved, as P',
    'Orthophosphate, Dissolved': 'Orthophosphate, Dissolved, as P',
    'Orthophosphate as PO4, Dissolved': 'Orthophosphate, Dissolved, as PO4',
    'Soluble Reactive Phosphorus (SRP), Dissolved': 'Orthophosphate, Dissolved, as P',
    'Soluble Reactive Phosphorus (SRP), Dissolved, as P': 'Orthophosphate, Dissolved, as P',
    'Orthophosphate, Organic': 'Orthophosphate, Total, as P',
    'Sulfate': 'Sulfate, Total, as S',
    'Sulfate as S': 'Sulfate, Total, as S',
    'Sulfate, Total': 'Sulfate, Total, as S',
    'Sulfate as S, Total': 'Sulfate, Total, as S',
    'Sulfate, as SO4': 'Sulfate, Total, as SO4',
    'Sulfate as SO4': 'Sulfate, Total, as SO4',
    'Sulfate as SO4, Total': 'Sulfate, Total, as SO4',
    'Sulfate, Dissolved': 'Sulfate, Dissolved, as S',
    'Sulfate, Dissolved, as S': 'Sulfate, Dissolved, as S',
    'Sulfate as SO4, Dissolved': 'Sulfate, Dissolved, as SO4',
    'Sulfate, Dissolved, as SO4': 'Sulfate, Dissolved, as SO4',
    'Sulfate, Volatile': 'Sulfate, Volatile, as S',
    'Sulfate, InOrganic': 'Sulfate, Total, as S',
    'Sulfite, Total': 'Sulfite, Total, as S',
    'Sulfite, as SO3': 'Sulfite, Total, as SO3',
    'Sulfur': 'Sulfur, Total, as S',
    'Sulfur, Total': 'Sulfur, Total, as S',
    'Sulfur, Dissolved': 'Sulfur, Dissolved, as S',
    'Sulfur, Dissolved, as S': 'Sulfur, Dissolved, as S',
    'Temperature, Dissolved': 'Temperature',
    'Temperature, Total': 'Temperature',
    'Temperature, deg F': 'Temperature',
    'Total Solids': 'Solids, Total',
    'Total Solids, Total': 'Solids, Total',
    'Total Solids, Dissolved': 'Solids, Total Dissolved',
    'Total Suspended Solids': 'Solids, Total Suspended',
    'Total Suspended Solids, Dissolved': 'Solids, Total Suspended',
    'Total Suspended Solids, Total': 'Solids, Total Suspended',
    'Total Suspended Solids, as CaCO3': 'Solids, Total Suspended',
    'Total Solids, Volatile': 'Solids, Total Volatile',
    'Total Volatile Solids': 'Solids, Total Volatile',
    'Total Volatile Solids, Dissolved': 'Solids, Total Volatile',
    'Total Volatile Solids, Total': 'Solids, Total Volatile',
    'Total Volatile Solids, Volatile': 'Solids, Total Volatile',
    'Total Suspended Solids, Volatile': 'Solids, Total Volatile Suspended',
    'Volatile Suspended Solids': 'Solids, Total Volatile Suspended',
    'Volatile Suspended Solids, Dissolved': 'Solids, Total Volatile Suspended',
    'Volatile Suspended Solids, Total': 'Solids, Total Volatile Suspended',
    'Volatile Suspended Solids, Volatile': 'Solids, Total Volatile Suspended',
    'pH, Dissolved': 'pH',
    'pH, Total': 'pH',
}

# Replaced values in 'CharacteristicName' using the mapping
EPA_df['Parameter'] = EPA_df['Parameter'].replace(replace_mapping)

In [ ]:
# Cleaned up commas and spaces.
EPA_df['Parameter'] = EPA_df['Parameter'].str.replace('\s+', ' ').str.strip()

In [ ]:
# Printed unique 'Parameter' values to check for consistant naming.
unique_parameters = sorted(EPA_df['Parameter'].unique())
for parameter in unique_parameters:
    print(parameter)

These all look good.

In [ ]:
# Printed unique 'Parameter' values and original values to check naming.
grouped_df = EPA_df.groupby('Parameter').agg({
    'ParameterOrig': 'unique'
}).reset_index()
pd.set_option('display.max_colwidth', None)
grouped_df

These all look good.

In [ ]:
# Dropped parameter feature as they are no longer needed.
features_to_drop = [
    'CharacteristicName',
    'ResultSampleFractionText',
    'MethodSpecificationName'
    ]
EPA_df.drop(columns=features_to_drop, inplace=True)

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data_05.csv', index=False)

### Examined results columns

In [ ]:
# Read EPA_data_05 file into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_05.csv')

In [ ]:
# Duplicated and combined features related to results so I have information in one place for future reference.
EPA_df['ResultOrig'] = (
    EPA_df['ResultMeasureValue'].fillna('').astype(str) +
    ' ' +
    EPA_df['ResultMeasure/MeasureUnitCode'].fillna('') +
    ' ' +
    EPA_df['MeasureQualifierCode'].fillna('')
)

In [ ]:
# Deleted rows where 'MeasureQualifierCode' indicates that the results were rejected, failed, or contaminated by the labratory.

# List of conditions to delete
conditions_to_delete = ['ISP', 'SCF', 'R', 'FFD', 'N', 'SCX', 'F', 'CON']

# Converted 'MeasureQualifierCode' to string and then delete rows where it contains specified conditions as whole words
pattern = r'\b(?:' + '|'.join(conditions_to_delete) + r')\b'
EPA_df = EPA_df[~EPA_df['MeasureQualifierCode'].astype(str).str.contains(pattern, case=False, regex=True)]

In [ ]:
# Printed unique data types and their counts to see what's in the results column.

# Counted data types
data_type_counts = EPA_df['ResultMeasureValue'].apply(lambda x: type(x).__name__).value_counts()

# Printed the results
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: type(x).__name__) == data_type].nunique()
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_values}")

This feature shopuld contain no strings.

In [ ]:
# Converted values into floats while leaving those that couldn't be converted in place.

# Defined function
def convert_to_float(value):
    try:
        # Tries converting the value to float
        return float(value)
    except (ValueError, TypeError):
        # If conversion failed, returned the original value
        return value

# Applied function
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(convert_to_float)

In [ ]:
# Counted number of NaNs in results to see how many of the floats are NaNs.
nan_count = EPA_df['ResultMeasureValue'].isna().sum()
print(nan_count)

In [ ]:
# Printed 100 of the unique string values to see some of the values in reults that need to be fixed.
unique_string_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str))].unique()
print(unique_string_values[:200])

In [ ]:
# Checked to see if comments will explain the *s.

# Selected specific columns
selected_columns_df = EPA_df[['ResultMeasureValue', 'ResultLaboratoryCommentText']]

# Applied filters for not null 'ResultLaboratoryCommentText' and containing asterisk in 'ResultMeasureValue'
filtered_df = selected_columns_df[
    (EPA_df['ResultLaboratoryCommentText'].notnull()) &
    (EPA_df['ResultMeasureValue'].str.contains('\*'))
]

# Printed unique combinations of the two columns
unique_combinations = filtered_df.drop_duplicates()
pd.set_option('display.max_colwidth', None)
print(unique_combinations)

In [ ]:
# Checked to see if an other feature will explain the *s.

# Filtered dataframe for floats with *s.
filtered_df = EPA_df[EPA_df['ResultMeasureValue'].str.contains('\*.*\.', na=False)]
pd.set_option('display.max_columns', None)
filtered_df

The comments and other features did not help determine the meaning of the astricks so I assumed the values are fine to use.

In [ ]:
# Removed the *s from the values.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: str(x).replace('*', '') if isinstance(x, str) else x)

In [ ]:
# Printed unique combinations of values in 'ResultMeasureValue' that contain '<' and 'ResultLaboratoryCommentText' to see if the comments explain anything.

# Filtered rows where 'ResultMeasureValue' contains '<'
filtered_df = EPA_df[EPA_df['ResultMeasureValue'].str.contains('<', case=False, na=False)]

# Printed unique and sorted combinations of 'ResultMeasureValue' and 'ResultLaboratoryCommentText'
unique_combinations = filtered_df[['ResultMeasureValue', 'ResultLaboratoryCommentText']].drop_duplicates()
unique_combinations = unique_combinations.sort_values(by='ResultMeasureValue')

for index, row in unique_combinations.iterrows():
    print(row['ResultMeasureValue'], row['ResultLaboratoryCommentText'])

No helpful comments.

In [ ]:
# Converted values starting with < to half the value. I am assuming the number is the minimum detection limit (MDL). It's common practice to devide the MDL by two so you have a number for analysis.

# Custom function to handle special cases
def process_value(value):
    if pd.notna(value):
        if isinstance(value, str):
            if value.startswith('<L'):
                # Leave '<L' as is
                return value
            elif value.startswith('<'):
                # Attempt to convert the rest to a float after removing '<'
                try:
                    return float(value[1:]) / 2
                except ValueError:
                    # If the conversion fails, return value
                    return value
            else:
                # If the value is a string but doesn't start with '<', return original value
                return value
        else:
            # If the value is not a string, return original value
            return value
    else:
        return value
    
# Applied the custom function to 'ResultMeasureValue'
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(process_value)

In [ ]:
# Replaced ".." in string values with ".". Assumed these are data entry errors.

# Defined function
def clean_and_convert(value):
    if isinstance(value, str):
        cleaned_value = value.replace('..', '.')
        return cleaned_value
    else:
        return value

# Applied function
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(clean_and_convert)

In [ ]:
# Converted values into floats while leaving those that couldn't be converted in place.

# Applied function
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(convert_to_float)

In [ ]:
# Printed 100 of the unique string values to see some of the values in reults that need to be fixed.
unique_string_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str))].unique()
print(unique_string_values[:200])

In [ ]:
# Printed unique values in the results that contain ':' and their count. 
detect_values = EPA_df[EPA_df['ResultMeasureValue'].str.contains(':', case=False, na=False)]['ResultMeasureValue'].unique()
for value in detect_values:
    print(value)

In [ ]:
# Checked to see if any feature will explain the :s.

# Filtered dataframe for :s.
filtered_df = EPA_df[EPA_df['ResultMeasureValue'].str.contains(':', na=False)]
pd.set_option('display.max_columns', None)
filtered_df

It appears that the results that look like times were incorrectly formatted in the source excel book before being uploaded to the EPA database. Therefore, the numbers retained the incorrect time format and I had to convert them back into the correct numbers the same way that Excel changes between time to number formats.

In [ ]:
# Converted string time value to float number.

# Custom conversion function for time strings and floats to numbers
def convert_value_to_number(value):
    try:
        if pd.notna(value):
            if isinstance(value, str) and ':' in value:
                time_obj = pd.to_datetime(value, format='%I:%M:%S %p').time()
                return (time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second) / 86400
            else:
                return float(value)
        else:
            return value
    except ValueError:
        return value

# Apply the custom conversion function to 'ResultMeasureValue'
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: convert_value_to_number(x))

In [ ]:
# Printed values from 'ResultMeasureValue' containing 'j'.
values_with_j = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].astype(str).str.contains('j', case=False, na=False)].tolist()
print(values_with_j)

In [ ]:
# Removed 'J' from values in 'ResultMeasureValue'. A 'J' qualifier tupicall means that the result was calculated by the lab rather than directly measured. I'm assuming the data is fine to use.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: str(x).replace('J', '') if isinstance(x, str) else x)

In [ ]:
# Checked to see if comments will explain non-numeric results.

# Filtered ResultMeasureValue for values not containing any numbers
filtered_df = EPA_df[~EPA_df['ResultMeasureValue'].astype(str).str.contains('\d', case=False, na=False, regex=True)]

# Printed unique combinations of ResultMeasureValue and ResultLaboratoryCommentText
unique_combinations = filtered_df[['ResultMeasureValue', 'ResultLaboratoryCommentText']].drop_duplicates().sort_values(by=['ResultMeasureValue', 'ResultLaboratoryCommentText'])

for index, row in unique_combinations.iterrows():
    print(row['ResultMeasureValue'], row['ResultLaboratoryCommentText'])

The comments explained some of the NaNs.

In [ ]:
# Replaced specified values in the 'ResultMeasureValue' column that indicate that the result was below the detection limit with '<L'. This was a place holder for future calcs. 
values_to_replace = ['<LOD', 'BLD', 'BPQL', 'MDP', 'ND', 'Non-detect', 'Not Detected',
                     'Not detected', 'Present <QL', 'Present Below Quantification Limit',
                     'Trace', 'Mdp', 'N.d.', 'Nd']

EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].replace(values_to_replace, '<L')

In [ ]:
# Replaced string values without numbers (excluding '<L') with NaN.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: np.nan if pd.isna(x) else (x if any(char.isdigit() for char in str(x)) or '<L' in str(x) else np.nan))

In [ ]:
# Checked incorrectly entered temperatures.

# Filtered ResultMeasureValue for values containing "F" or "C"
filtered_df = EPA_df[EPA_df['ResultMeasureValue'].astype(str).str.contains('F|C', case=False, na=False, regex=True)]

# Printed unique combinations of ResultMeasureValue and ResultMeasure/MeasureUnitCode
unique_combinations = filtered_df[['ResultMeasureValue', 'ResultMeasure/MeasureUnitCode']].drop_duplicates().sort_values(by=['ResultMeasureValue', 'ResultMeasure/MeasureUnitCode'])

for index, row in unique_combinations.iterrows():
    print(row['ResultMeasureValue'], row['ResultMeasure/MeasureUnitCode'])

In [ ]:
# Changed 'C' to 'F'.

# Values to find in 'ResultMeasureValue'
values_to_find = ['35 F', '36 F', '39 F', '40 F', '40F', '42 F', '43 F', '45 F', '46 F', '47 F',
                  '48 F', '49 F', '50 F', '51 F', '52 F', '53 F', '54 F', '55 F', '56 F', '57 F',
                  '58 F', '59 F', '60 F', '63 F', '64 F', '64.5F', '65 F', '66 F', '67 F', '68 F',
                  '69 F', '70 F', '71 F', '72 F', '73 F', '74 F', '75 F', '76 F', '77 F', '78 F',
                  '79 F', '80 F', '80F', '82F', '94 F']

# Replaced the corresponding values in 'ResultMeasure/MeasureUnitCode' with 'deg F'
EPA_df.loc[EPA_df['ResultMeasureValue'].isin(values_to_find), 'ResultMeasure/MeasureUnitCode'] = 'deg F'

In [ ]:
# Replaced mis-entered temperature values.

values_to_replace = {
    '27.4 C': '27.4',
    '27.6 C': '27.6',
    '28.1 C': '28.1',
    '35 F': '35',
    '36 F': '36',
    '39 F': '39',
    '40 F': '40',
    '40F': '40',
    '42 F': '42',
    '43 F': '43',
    '45 F': '45',
    '46 F': '46',
    '47 F': '47',
    '48 F': '48',
    '49 F': '49',
    '50 F': '50',
    '51 F': '51',
    '52 F': '52',
    '53 F': '53',
    '54 F': '54',
    '55 F': '55',
    '56 F': '56',
    '57 F': '57',
    '58 F': '58',
    '59 F': '59',
    '60 F': '60',
    '63 F': '63',
    '64 F': '64',
    '64.5F': '64.5',
    '65 F': '65',
    '66 F': '66',
    '67 F': '67',
    '68 F': '68',
    '69 F': '69',
    '70 F': '70',
    '71 F': '71',
    '72 F': '72',
    '73 F': '73',    
    '74 F': '74',
    '75 F': '75',
    '76 F': '76',
    '77 F': '77',
    '78 F': '78',
    '79 F': '79',
    '80 F': '80',
    '80F': '80', 
    '82F': '82',
    '94 F': '94',
}

# Replaced the values in 'ResultMeasureValue'
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].replace(values_to_replace, regex=True)

In [ ]:
# Converted values into floats while leaving those that couldn't be converted in place.

# Applied function
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(convert_to_float)

In [ ]:
# Printed 100 of the unique string values to see some of the values in reults that need to be fixed.
unique_string_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str))].unique()
print(unique_string_values[:200])

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data_06.csv', index=False)

In [ ]:
# Read EPA_data_06 file into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_06.csv')

In [ ]:
# Printed list of values that contain '>'.
unique_values_with_greater = EPA_df['ResultMeasureValue'].astype(str)[EPA_df['ResultMeasureValue'].astype(str).str.contains('>')].unique()
print(list(unique_values_with_greater))

In [ ]:
# Replaced '>22 (OUT OF PROBE RANGE)' with '>22'.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: str(x).replace('>22 (OUT OF PROBE RANGE)', '22') if isinstance(x, str) else x)

In [ ]:
# Removed '>' from strings that begin with '>'. The real value is larger than the number but I don't know by how much so I defaulted to the minimum.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: x[1:] if isinstance(x, str) and x.startswith('>') else x)

In [ ]:
# Printed list of values that contain 'Invalid '.
unique_values_with_invalid = EPA_df['ResultMeasureValue'].astype(str)[EPA_df['ResultMeasureValue'].astype(str).str.contains('Invalid ')].unique()
print(list(unique_values_with_invalid))

In [ ]:
# Removed 'Invalid ' from strings that contain 'Invalid '. A value was produced so I'll use it.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: str(x).replace('Invalid ', '') if isinstance(x, str) else x)

In [ ]:
# Printed values that begin with '.'
dot_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].str.startswith('.').fillna(False)]
print(dot_values)

In [ ]:
# Removed '.' from strings that begin with '.'. I assumed these to be data entry errors.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: x[1:] if isinstance(x, str) and x.startswith('.') else x)

In [ ]:
# Printed values that end with '.'
dot_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].str.endswith('.').fillna(False)]
print(dot_values)

In [ ]:
# Removed '.' from strings that end with '.'.  I assumed these to be data entry errors.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: x[:-1] if isinstance(x, str) and x.endswith('.') else x)

In [ ]:
# Printed list of values that contain '='.
unique_values_with_equals = EPA_df['ResultMeasureValue'].astype(str)[EPA_df['ResultMeasureValue'].astype(str).str.contains('=')].unique()
print(list(unique_values_with_equals))

In [ ]:
# Removed '=' from strings that begin with '='. I assumed these to be data entry errors.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: x[1:] if isinstance(x, str) and x.startswith('=') else x)

In [ ]:
# Removed 'ND LOD=' and divided number by 2 becasue the number provided is the detection limit.

# Custom function to handle special cases
def process_value(value):
    if pd.notna(value):
        if isinstance(value, str):
            if value.startswith('ND LOD='):
                # Attempt to convert the rest to a float after removing 'ND LOD=' and then divide by 2
                try:
                    return float(value.replace('ND LOD=', '')) / 2
                except ValueError:
                    # If the conversion fails, return value
                    return value
            else:
                # If the value is a string but doesn't start with 'ND LOD=', return original value
                return value
        else:
            # If the value is not a string, return original value
            return value
    else:
        return value

# Applied the custom function to 'ResultMeasureValue'
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(process_value)

In [ ]:
# Printed list of values that contain ' at  '.
unique_values_with_at = EPA_df['ResultMeasureValue'].astype(str)[EPA_df['ResultMeasureValue'].astype(str).str.contains(' at ', case=False)].unique()
print(list(unique_values_with_at))

In [ ]:
# Removed 'at' followed by a number from 'ResultMeasureValue'. This information is not useful.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: re.sub(r'\s*AT\s*\d+', '', str(x)) if isinstance(x, str) else x)

In [ ]:
# Printed list of values that contain ','.
unique_values_with_comma = EPA_df['ResultMeasureValue'].astype(str)[EPA_df['ResultMeasureValue'].astype(str).str.contains(',')].unique()
print(list(unique_values_with_comma))

In [ ]:
# Replaced values with commas. I assume these to be typos.
values_to_replace = {
    '13,75': '13.75',
    '0,09': '0.09',
    '3,5': '3.5',
    '32,7': '32.7',
}

# Replaced the values in 'ResultMeasureValue'
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].replace(values_to_replace, regex=True)

In [ ]:
# Removed ',' from strings that contain ','.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)

In [ ]:
# Printed list of values that contains '[' or ']'.
unique_values_with_bracket = EPA_df['ResultMeasureValue'].astype(str)[EPA_df['ResultMeasureValue'].astype(str).str.contains('[\[\]]')].unique()
print(list(unique_values_with_bracket))

In [ ]:
# Removed brackets from strings.  I assumed these to be data entry errors.
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(lambda x: x.replace('[', '').replace(']', '') if isinstance(x, str) else x)

In [ ]:
# Converted values into floats while leaving those that couldn't be converted in place.

# Applied function
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(convert_to_float)

In [ ]:
# Printed the unique string values to see the rest of the values in reults that need to be fixed.
unique_string_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str))].unique()
print(unique_string_values)

In [ ]:
# Printed remaining string 'ResultMeasureValue' with select other coulumns to see if they can explain the value.
selected_columns = [
    'ResultDetectionConditionText',
    'Parameter',
    'ResultMeasureValue',
    'ResultMeasure/MeasureUnitCode',
    'MeasureQualifierCode',
    'ResultAnalyticalMethod/MethodIdentifier',
    'ResultLaboratoryCommentText'
]

# Filtered data
filtered_df = EPA_df[EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str) and x != '<L')]
filtered_df_selected_columns = filtered_df[selected_columns]

# Sorted by 'ResultMeasureValue'
sorted_df = filtered_df_selected_columns.sort_values(by='ResultMeasureValue')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
sorted_df

In [ ]:
# Deleted rows where I could not determine true 'ResultMeasureValue'.
values_to_delete = ["0.002'447.0.0", "1300(A)", "19-72", "300(A)", "300(A)0", "X2", "X3", "Z2", "86.2%"]
EPA_df = EPA_df[~EPA_df['ResultMeasureValue'].isin(values_to_delete)]

In [ ]:
# Changed 'ResultMeasureValue' where I could determine true value.

values_to_replace = {
    '0.014mg/L': '0.014', # Correct units already in data 
    '0.94mg/L': '0.94', # Correct units already in data 
    '1.14mg/L': '1.14', # Correct units already in data 
    '1.15µg/L': '1.15', # Correct units already in data 
    '10-15': '12.5', # Took mean of range
    '11.\n11.8': '11.8', # Value make sense for parameter
    '12 9': '12.9', # Value with decimal make sense for parameter
    '12/31/1899 5:52:48 AM': '1.25', # Format change
    '12/31/1899 9:04:00 AM': '1.38', # Format change
    '136.44% 12.134 mg': '12.134', # Kept DO in mg to match units
    '148`': '148', # Assumed to be a typo
    '16:19:12': '15.67', # Took mean of range
    '20\+': '20', # Assumed to be a typo
    '21.9\n21.9': '21.9', # Value make sense for parameter
    '21/1': '21.1', # Value make sense for parameter
    '22.56-23.89': '23.225', # Took mean of range
    '23.3\n23.3': '23.3', # Value make sense for parameter
    '23.6\n23.6': '23.6', # Value make sense for parameter
    '2O2': '202', # Assumed to be a typo
    '36\n36': '36', # Value make sense for parameter
    '4.8\n4.8': '4.8', # Value make sense for parameter
    '5 .9': '5.9', # Assumed to be a typo
    '6\n6': '6', # Value make sense for parameter
    '6.4\n6.4': '6.4', # Value make sense for parameter
    '60.26% 5.85 mg': '5.85', # Kept DO in mg to match units
    '7\n27.2': '27.2', # Value make sense for parameter
    '7.27.4': '7.3', # Took mean of range
    '7.57.2': '7.35', # Took mean of range
    '7.77.3': '7.5', # Took mean of range
    '75.5% 8.06mg/l': '8.06', # Kept DO in mg to match units
    '79.6% 6.69 mg': '6.69', # Kept DO in mg to match units
    '8.48\+': '8.48', # Assumed to be a typo
    'E11.4': '11.4', # Assumed to be a typo
    'O.3': '0.3', # Assumed to be a typo
}

# Replace the values in 'ResultMeasureValue'
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].replace(values_to_replace, regex=True)

In [ ]:
# Converted values into floats while leaving those that couldn't be converted in place.

# Applied function
EPA_df['ResultMeasureValue'] = EPA_df['ResultMeasureValue'].apply(convert_to_float)

In [ ]:
# Printed the unique string values to see the rest of the values in reults that need to be fixed.
unique_string_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str))].unique()
print(unique_string_values)

In [ ]:
# Counted number of NaNs in results.
nan_count = EPA_df['ResultMeasureValue'].isna().sum()
print(nan_count)

Need to fill in the 'ResultMeasureValue' NaNs with values.

In [ ]:
# Checked to see if 'ResultDetectionConditionText' will explain non-numeric results.

# Filter ResultMeasureValue for values not containing any numbers
filtered_df = EPA_df[~EPA_df['ResultMeasureValue'].astype(str).str.contains('\d', case=False, na=False, regex=True)]

# Print unique combinations of ResultMeasureValue and ResultDetectionConditionText
unique_combinations = filtered_df[['ResultMeasureValue', 'ResultDetectionConditionText']].drop_duplicates().sort_values(by=['ResultMeasureValue', 'ResultDetectionConditionText'])

for index, row in unique_combinations.iterrows():
    print(row['ResultMeasureValue'], row['ResultDetectionConditionText'])

Yes, 'ResultDetectionConditionText' will explain some non-numeric results.

In [ ]:
# Replaced NaNs in 'ResultMeasureValue' with '<L' for values from 'ResultDetectionConditionText' that all mean less than limit. '<L' is a place holder for future calculation.

# List of conditions
conditions = [
    'Present Below Quantification Limit',
    'Not Detected',
    'Below Method Detection Limit',
    'Detected Not Quantified',
    'Below Detection Limit',
    '*Non-detect',
    '*NOT DETECTED',
    '*Present <QL',
    'Not Detected at Reporting Limit',
    'Below Reporting Limit',
    '*Present < QL',
    '*Present<QL',
    '*Present',
    'Not Detected at Detection Limit'
]

# Filled NaN values in 'ResultMeasureValue' based on the specified conditions
EPA_df['ResultMeasureValue'] = np.where(
    EPA_df['ResultMeasureValue'].isna() & EPA_df['ResultDetectionConditionText'].isin(conditions),
    '<L',
    EPA_df['ResultMeasureValue']
)

In [ ]:
# Checked to see if 'MeasureQualifierCode' will explain non-numeric results.

# Filtered ResultMeasureValue for values not containing any numbers
filtered_df = EPA_df[~EPA_df['ResultMeasureValue'].astype(str).str.contains('\d', case=False, na=False, regex=True)]

# Printed unique combinations of ResultMeasureValue and MeasureQualifierCode
unique_combinations = filtered_df[['ResultMeasureValue', 'MeasureQualifierCode']].drop_duplicates().sort_values(by=['ResultMeasureValue', 'MeasureQualifierCode'])

for index, row in unique_combinations.iterrows():
    print(row['ResultMeasureValue'], row['MeasureQualifierCode'])

Yes, 'MeasureQualifierCode' will explain some non-numeric results.

In [ ]:
# Replaced NaNs in 'ResultMeasureValue' with '<L' for values from 'MeasureQualifierCode' that all mean less than limit. '<L' was a place holder for future calculation.

# List of conditions to fill NaN values with '<L'
conditions_to_fill = ['J', 'B', 'I', 'Q', 'R', 'D', 'DL', 'FDL']

# Filled NaN values in 'ResultMeasureValue' with '<L' when 'MeasureQualifierCode' contains specified conditions
pattern = r'\b(?:' + '|'.join(conditions_to_fill) + r')\b'
EPA_df['ResultMeasureValue'] = np.where(
    EPA_df['ResultMeasureValue'].isna() & EPA_df['MeasureQualifierCode'].astype(str).str.contains(pattern, case=False, regex=True),
    '<L',
    EPA_df['ResultMeasureValue']
)

In [ ]:
# Checked to see if comments will explain non-numeric results.

# Filtered ResultMeasureValue for values not containing any numbers
filtered_df = EPA_df[~EPA_df['ResultMeasureValue'].astype(str).str.contains('\d', case=False, na=False, regex=True)]

# Printed unique combinations of ResultMeasureValue and ResultLaboratoryCommentText
unique_combinations = filtered_df[['ResultMeasureValue', 'ResultLaboratoryCommentText']].drop_duplicates().sort_values(by=['ResultMeasureValue', 'ResultLaboratoryCommentText'])

for index, row in unique_combinations.iterrows():
    print(row['ResultMeasureValue'], row['ResultLaboratoryCommentText'])

Yes, 'ResultLaboratoryCommentText' will explain some non-numeric results.

In [ ]:
# Replaced NaNs in 'ResultMeasureValue' with '<L' for values from 'ResultLaboratoryCommentText' that all mean less than limit. '<L' was a place holder for future calculation.
mask = EPA_df['ResultLaboratoryCommentText'].str.contains('below', case=False, na=False)
EPA_df.loc[mask, 'ResultMeasureValue'].fillna('<L', inplace=True)
EPA_df['ResultMeasureValue'].replace({None: np.nan}, inplace=True)

In [ ]:
# Printed unique data types and their counts to see what's in the results column.

# Counted data types
data_type_counts = EPA_df['ResultMeasureValue'].apply(lambda x: type(x).__name__).value_counts()

# Printed the results
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: type(x).__name__) == data_type].nunique()
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_values}")

Now all infomation about results below detection limit are in the results column. 

In [ ]:
# Counted number of NaNs in results.
nan_count = EPA_df['ResultMeasureValue'].isna().sum()
print(nan_count)

In [ ]:
# Printed the unique string values to see the rest of the values in results that need to be fixed.
unique_string_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: isinstance(x, str))].unique()
print(unique_string_values)

In [ ]:
# Deleted all rows where the result is NaN. Nothing else can be done to fill in these values so the rows are not useful.
EPA_df = EPA_df.dropna(subset=['ResultMeasureValue'])

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data_07.csv', index=False)

### Examined units column

In [ ]:
# Read EPA_data_07 file into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_07.csv')

In [ ]:
# Replaced 'ResultMeasure/MeasureUnitCode' with NaN when 'ResultMeasureValue' is NaN or '<L'. Having units are pointless when there is no result value.
mask = EPA_df['ResultMeasureValue'].isna() | (EPA_df['ResultMeasureValue'] == '<L')
EPA_df.loc[mask, 'ResultMeasure/MeasureUnitCode'] = np.NaN

In [ ]:
# Printed unique values in 'ResultDepthHeightMeasure/MeasureUnitCode' to list all units.
unique_values_list = sorted(map(str, EPA_df['ResultMeasure/MeasureUnitCode'].unique()))
for value in unique_values_list:
    print(value)

In [ ]:
# Replaced mis-entered and equivalent units for consistancey.

values_to_replace = {
    'mg/l': 'mg/L',
    'mg/l as N': 'mg/L',
    'mg/l asNO2': 'mg/L',
    'mg/l asNO3': 'mg/L',
    'mg/l as P': 'mg/L',
    'mg/l asPO4': 'mg/L',
    'mg/l CaCO3': 'mg/L',
    'mg/l CaCO3**': 'mg/L',
    'mg/l NH4': 'mg/L',
    'mg/l NO3': 'mg/L',
    'mg/l PO4': 'mg/L',
    'mg N/l******': 'mg/L',
    'ug/l': 'ug/L',
    'ug/l as P': 'ug/L',
    'ug/L as P': 'ug/L',
    'uS/cm @25C': 'uS/cm',
    '% saturatn**': '%',
    'ppm': 'mg/L',
    'ppb': 'ug/L',
    'ppt': 'ng/L',
    'mg/kg': 'mg/L',
    'mg/g': 'g/L',
    'uS/cm': 'umho/cm',
    'mg/m3': 'ug/L',
    'mg/m2': 'm3',
    '% by wt': '%',
    '% solids': '%',
    'nu': 'std units',
    'units/cm': 'std units',
}

# Iterated through the dictionary and replace values based on exact matches
for value, replacement in values_to_replace.items():
    EPA_df.loc[EPA_df['ResultMeasure/MeasureUnitCode'] == value, 'ResultMeasure/MeasureUnitCode'] = replacement

In [ ]:
# Printed information about each unit to help decide how they should be edited.
for parameter_value in EPA_df['Parameter'].unique():
    subset_df = EPA_df[EPA_df['Parameter'] == parameter_value]
    unit_counts = subset_df['ResultMeasure/MeasureUnitCode'].value_counts()
    
    print(f"Parameter: {parameter_value}")
    
    for unit_code, count in unit_counts.items():
        values_numeric = pd.to_numeric(subset_df[subset_df['ResultMeasure/MeasureUnitCode'] == unit_code]['ResultMeasureValue'], errors='coerce')
        min_val = values_numeric.min()
        max_val = values_numeric.max()
        median_val = values_numeric.median()
        
        print(f"  Unit: {unit_code}")
        print(f"    Count: {count}")
        print(f"    Minimum: {min_val}")
        print(f"    Maximum: {max_val}")
        print(f"    Median: {median_val}")
    
    print()

Many units don't match the parameter and need to be changed of deleted. Also, many need to be converted to another unit. Many units have a low count and don't make sense for their respective parameter or aren't worth the effort to edit.

In [ ]:
# Deleted rows for units counts equal to or less than 10.

# Iterated through unique parameter values
for parameter_value in EPA_df['Parameter'].unique():
    
    # Created a subset DataFrame for the current parameter
    subset_df = EPA_df[EPA_df['Parameter'] == parameter_value]
    
    # Calculated the count of each unit code in the 'ResultMeasure/MeasureUnitCode' column
    unit_counts = subset_df['ResultMeasure/MeasureUnitCode'].value_counts()
    
    # Got unit codes with count <= 10
    units_to_delete = unit_counts[unit_counts <= 10].index
    
    # Identified rows to delete
    rows_to_delete = subset_df[subset_df['ResultMeasure/MeasureUnitCode'].isin(units_to_delete)].index
    
    # Checked if there are rows to delete
    if not rows_to_delete.empty:
        # Delete the rows from the DataFrame
        EPA_df.drop(rows_to_delete, inplace=True)
        
    # Ran checkpoints for debugging
    print(f"Processing parameter: {parameter_value}")

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data_08.csv', index=False)

In [ ]:
# Read EPA_data_08 file into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_08.csv')

In [ ]:
# Fixed a parameter naming error I missed before.
EPA_df['Parameter'] = EPA_df['Parameter'].replace('Dissolved Oxygen Saturation', 'Dissolved Oxygen, Saturation')

In [ ]:
# Fixed dissolved oxygen naming errors I missed before.
condition = (EPA_df['Parameter'] == 'Dissolved Oxygen') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%')
EPA_df.loc[condition, 'Parameter'] = 'Dissolved Oxygen, Saturation'

In [ ]:
# Fixed dissolved oxygen naming errors I missed before.
condition = (EPA_df['Parameter'] == 'Dissolved Oxygen, Saturation') & (EPA_df['ResultMeasure/MeasureUnitCode'] == 'mg/L')
EPA_df.loc[condition, 'Parameter'] = 'Dissolved Oxygen'

In [ ]:
# Deleted an unneeded parameter I missed before.
EPA_df.drop(EPA_df[EPA_df['Parameter'] == 'Light, Transmissivity'].index, inplace=True)

In [ ]:
# Deleted rows with units for specific parameters.
conditions = (
    (EPA_df['Parameter'] == 'Temperature') & (EPA_df['ResultMeasure/MeasureUnitCode'] == 'ft') | # This unit doesn't make sense and values don't match for parameter.
    (EPA_df['Parameter'] == 'Temperature') & (EPA_df['ResultMeasure/MeasureUnitCode'] == 'm') | # This unit doesn't make sense and values don't match for parameter.
    (EPA_df['Parameter'] == 'Nitrogen, Total, as N') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%') | # This unit doesn't make sense and values don't match for parameter.
    (EPA_df['Parameter'] == 'Chlorophyll a, Total') & (EPA_df['ResultMeasure/MeasureUnitCode'] == 'volts') | # This unit doesn't make sense and values don't match for parameter.
    (EPA_df['Parameter'] == 'pH') & (EPA_df['ResultMeasure/MeasureUnitCode'] == 'mV') | # Don't have information to create a calibration curve needed for conversion to standard units.
    (EPA_df['Parameter'] == 'Solids, Total Suspended') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%') | # Don't know volume of water for conversion to mg/L.
    (EPA_df['Parameter'] == 'Solids, Total Volatile Suspended') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%') | # Don't know volume of water for conversion to mg/L.
    (EPA_df['Parameter'] == 'Solids, Total Volatile') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%') | # Don't know volume of water for conversion to mg/L.
    (EPA_df['Parameter'] == 'Solids, Total') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%') | # Don't know volume of water for conversion to mg/L.
    (EPA_df['Parameter'] == 'Carbon, Total') & (EPA_df['ResultMeasure/MeasureUnitCode'] == '%') # Don't know volume of water for conversion to mg/L.
)

EPA_df = EPA_df.loc[~conditions]

In [ ]:
# Deleted rows with specific units.
units_to_delete = [
   'tons/day', # Don't know flow rate for conversion to mg/L.
    'FNU', # Can't be converted to mg/L.
    'kg', # Don't know volume of water for conversion to mg/L.
    'gpg', # Unknown unit
    'RFU', # Don't know calibration coefficients from calibration curve for conversion to mg/L.
    '#/L', # Don't know chlorophyll-a concentration per cell or conversion to mg/L.
    'IVFU', # Don't know calibration coefficients from calibration curve for conversion to mg/L.
    'umol', # Don't know volume of water for conversion to mg/L.
    'g', # Don't know volume of water for conversion to mg/L.
    'm3' # Don't know weight for conversion to mg/L.
]
EPA_df = EPA_df.loc[~EPA_df['ResultMeasure/MeasureUnitCode'].isin(units_to_delete)]

In [ ]:
# Changed unit names for specific parameters. The original unit did not make sense for the parameter but the values alligned with the new unit.

# Dictionary mapping conditions to the desired changes
conditions = {
    ('Temperature', 'mg/L'): ('deg C',),
    ('pH', 'umho/cm'): ('std units',),
    ('pH', 'mg/L'): ('std units',),
    ('Dissolved Oxygen', 'umho/cm'): ('mg/L',),
    ('Dissolved Oxygen', 'ug/L'): ('mg/L',),
    ('Dissolved Oxygen', 'cm3/L'): ('mg/L',),
    ('Conductivity, Total', 'mg/L'): ('umho/cm',),
    ('Conductivity, Total', 'ug/L'): ('umho/cm',),
    ('Conductivity, Total', 'std units'): ('umho/cm',),
    ('Conductivity, Total', 'umho'): ('umho/cm',),
    ('Oxidation Reduction Potential (ORP), Total', 'mg/L'): ('volts',)
}

# Applied changes using conditions
for (param, unit), (new_unit,) in conditions.items():
    mask = (EPA_df['Parameter'] == param) & (EPA_df['ResultMeasure/MeasureUnitCode'] == unit)
    EPA_df.loc[mask, 'ResultMeasure/MeasureUnitCode'] = new_unit


In [ ]:
# Changed unit names. The original unit did not make sense for any parameter with which it was associated but the values alligned with the new unit.

# Mapping of values to be replaced
replace_mapping = {
    'Deg': 'deg C',
    'count': 'std units',
    'Mole/L': 'std units',
    'mg': 'mg/L',
    'm/y': 'mV',
    'S/m': 'umho/cm'
}

# Applied replacements in the DataFrame
EPA_df['ResultMeasure/MeasureUnitCode'] = EPA_df['ResultMeasure/MeasureUnitCode'].replace(replace_mapping)

In [ ]:
# Converted values and units for specific parameters to correct values and units.

# Function to convert 'Chloride, Dissolved' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_chloride_dissolved(umol_per_L):
    return umol_per_L * 35.453

# Function to convert 'Nitrogen, Dissolved, as N' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_nitrogen_dissolved(umol_per_L):
    return umol_per_L * 0.1401

# Function to convert 'Nitrogen (NO2 & NO3), Dissolved, as N' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_NO2NO3_dissolved(umol_per_L):
    return umol_per_L * 0.1401

# Function to convert 'Sulfate, Dissolved, as S' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_sulfate_dissolved(umol_per_L):
    return umol_per_L * 0.3207

# Function to convert 'Nitrate (NO3), Dissolved, as N' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_nitrate_dissolved(umol_per_L):
    return umol_per_L * 0.1401

# Function to convert 'Nitrogen (NO2, NO3, & NH3), Total, as N' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_nitrogen_total_as_N(umol_per_L):
    return umol_per_L * 0.1401

# Function to convert 'Ammonia (NH3), Dissolved, as NH3' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_ammonia_dissolved(umol_per_L):
    return umol_per_L * 0.17031

# Function to convert 'Nitrite (NO2), Dissolved, as NO2' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_nitrite_dissolved(umol_per_L):
    return umol_per_L * 0.46005

# Function to convert 'Nitrate (NO3), Dissolved, as NO3' from umol/L to mg/L. Used parameter molecular weight but modified micro and milli to produce values that made sense.
def convert_nitrate_dissolved_as_N(umol_per_L):
    return umol_per_L * 0.620049

# Function to convert 'Chloride, Dissolved' from mmol/L to mg/L. Used parameter molecular weight but modified milli to produce values that made sense.
def convert_chloride_dissolved_mmol(mmol_per_L):
    return mmol_per_L * 0.35453

# Function to convert 'Chloride, Total' from ueq/L to mg/L. Used parameter molecular weight divided by it's valence but modified micro and milli to produce values that made sense.
def convert_chloride_total_ueq(ueq_per_L):
    return ueq_per_L * 0.35453 / 1

# Function to convert 'Chloride, Dissolved' from ueq/L to mg/L. Used parameter molecular weight divided by it's valence but modified micro and milli to produce values that made sense.
def convert_chloride_dissolved_ueq(ueq_per_L):
    return ueq_per_L * 35.453 / 1

# Function to convert 'Alkalinity, Total, as CaCO3' from ueq/L to mg/L. Used parameter molecular weight divided by it's valence times 2 for carbonate and bicarbonate contributions but modified micro and milli to produce values that made sense.
def convert_alkalinity_total(ueq_per_L):
    return ueq_per_L * 0.61006 / 1 * 2

# Function to convert 'Sulfate, Dissolved, as SO4' from ueq/L to mg/L. Used parameter molecular weight divided by it's valence but modified micro and milli to produce values that made sense.
def convert_sulfate_dissolved_so4(ueq_per_L):
    return ueq_per_L * 0.960636 / 2

# Function to convert 'Nitrate (NO3), Total, as NO3' from ueq/L to mg/L. Used parameter molecular weight divided by it's valence but modified micro and milli to produce values that made sense.
def convert_nitrate_total_as_NO3(ueq_per_L):
    return ueq_per_L * 0.620045 / 1

# Dictionary mapping parameters to conversion functions and new unit codes
conversion_functions = {
    'Chloride, Dissolved': (convert_chloride_dissolved, 'mg/L'),
    'Nitrogen, Dissolved, as N': (convert_nitrogen_dissolved, 'mg/L'),
    'Nitrogen (NO2 & NO3), Dissolved, as N': (convert_NO2NO3_dissolved, 'mg/L'),
    'Sulfate, Dissolved, as S': (convert_sulfate_dissolved, 'mg/L'),
    'Nitrate (NO3), Dissolved, as N': (convert_nitrate_dissolved, 'mg/L'),
    'Ammonia (NH3), Dissolved, as NH3': (convert_ammonia_dissolved, 'mg/L'),
    'Nitrite (NO2), Dissolved, as NO2': (convert_nitrite_dissolved, 'mg/L'),
    'Nitrate (NO3), Dissolved, as NO3': (convert_nitrate_dissolved_as_N, 'mg/L'),
    'Nitrogen (NO2, NO3, & NH3), Total, as N': (convert_nitrogen_total_as_N, 'mg/L'),
    'Chloride, Dissolved': (convert_chloride_dissolved_mmol, 'mg/L'),
    'Chloride, Total': (convert_chloride_total_ueq, 'mg/L'),
    'Chloride, Dissolved': (convert_chloride_dissolved_ueq, 'mg/L'),
    'Alkalinity, Total, as CaCO3': (convert_alkalinity_total, 'mg/L'),
    'Sulfate, Dissolved, as SO4': (convert_sulfate_dissolved_so4, 'mg/L'),
    'Nitrate (NO3), Total, as NO3': (convert_nitrate_total_as_NO3, 'mg/L')
}

# Applied conversions to the DataFrame
for parameter, (conversion_function, new_unit_code) in conversion_functions.items():
    mask = EPA_df['Parameter'] == parameter
    numeric_mask = pd.to_numeric(EPA_df.loc[mask, 'ResultMeasureValue'], errors='coerce').notna()
    
    # Handled "<L" values separately
    below_limit_mask = EPA_df.loc[mask, 'ResultMeasureValue'] == '<L'
    
    # Stored original unit codes for comparison
    original_unit_codes = EPA_df.loc[mask, 'ResultMeasure/MeasureUnitCode'].copy()
    
    # Applied the conversion function to numeric values
    converted_values = pd.to_numeric(EPA_df.loc[mask & ~below_limit_mask, 'ResultMeasureValue'], errors='coerce').apply(conversion_function)
    
    # Checked if conversion occurred and update the values and unit code accordingly
    conversion_occurred_mask = ~pd.isna(converted_values)
    EPA_df.loc[mask & ~below_limit_mask, 'ResultMeasureValue'] = converted_values[conversion_occurred_mask]
    EPA_df.loc[mask & conversion_occurred_mask, 'ResultMeasure/MeasureUnitCode'] = new_unit_code

In [ ]:
# Converted values and units to correct values and units.

# Conversion functions
def convert_degF_to_degC(degF):
    return (degF - 32) * 5.0/9.0

def convert_ugL_to_mgL(ugL):
    return ugL / 1000.0

def convert_volts_to_mV(volts):
    return volts * 1000.0

def convert_mS_cm_to_umho_cm(mS_cm):
    return mS_cm * 1000.0

def convert_S_m_to_umho_cm(S_m):
    return S_m * 1000000.0

def convert_mho_cm_to_umho_cm(mho_cm):
    return mho_cm * 1000.0

def convert_mmhos_cm_to_umho_cm(mmhos_cm):
    return mmhos_cm * 1000.0

def convert_mS_m_to_umho_cm(mS_m):
    return mS_m * 1000.0

def convert_ft_to_m(ft):
    return ft * 0.3048

def convert_in_to_m(inch):
    return inch * 0.0254

# Dictionary mapping units to conversion functions
conversion_functions = {
    'deg F': (convert_degF_to_degC, 'deg C'),
    'ug/L': (convert_ugL_to_mgL, 'mg/L'),
    'volts': (convert_volts_to_mV, 'mV'),
    'mS/cm': (convert_mS_cm_to_umho_cm, 'umho/cm'),
    'S/m': (convert_S_m_to_umho_cm, 'umho/cm'),
    'mho/cm': (convert_mho_cm_to_umho_cm, 'umho/cm'),
    'mmhos/cm': (convert_mmhos_cm_to_umho_cm, 'umho/cm'),
    'mS/m': (convert_mS_m_to_umho_cm, 'umho/cm'),
    'ft': (convert_ft_to_m, 'm'),
    'in': (convert_in_to_m, 'm')
}

# Conversion functions
def convert_with_handling(value, conversion_function):
    try:
        return conversion_function(float(value))
    except ValueError:
        return value  # Return as is for non-numeric values

# Apply conversions to the DataFrame
for unit_code, (conversion_function, new_unit_code) in conversion_functions.items():
    if unit_code == '<L':
        continue  # Skip conversion for '<L'

    mask = EPA_df['ResultMeasure/MeasureUnitCode'] == unit_code
    EPA_df.loc[mask, 'ResultMeasureValue'] = EPA_df.loc[mask, 'ResultMeasureValue'].apply(lambda x: convert_with_handling(x, conversion_function))
    EPA_df.loc[mask, 'ResultMeasure/MeasureUnitCode'] = new_unit_code

In [ ]:
# Reprinted information about each unit to confirm changes.
for parameter_value in EPA_df['Parameter'].unique():
    subset_df = EPA_df[EPA_df['Parameter'] == parameter_value]
    unit_counts = subset_df['ResultMeasure/MeasureUnitCode'].value_counts()
    
    print(f"Parameter: {parameter_value}")
    
    for unit_code, count in unit_counts.items():
        values_numeric = pd.to_numeric(subset_df[subset_df['ResultMeasure/MeasureUnitCode'] == unit_code]['ResultMeasureValue'], errors='coerce')
        min_val = values_numeric.min()
        max_val = values_numeric.max()
        median_val = values_numeric.median()
        
        print(f"  Unit: {unit_code}")
        print(f"    Count: {count}")
        print(f"    Minimum: {min_val}")
        print(f"    Maximum: {max_val}")
        print(f"    Median: {median_val}")
    
    print()

Looks good. All parameters have one unit (except for those where all results ar '<L') and that unit is appropriate for the parameter.

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data_09.csv', index=False)

### Imputed half detection levels

In [ ]:
# Read EPA_data_09 file into pandas DataFrame.
EPA_df = pd.read_csv('/Users/carahcampini/Desktop/Capstone/Edited_Data/EPA_data_09.csv')

In [ ]:
# Counted number of '<L' values in resluts.
count_of_L_values = EPA_df['ResultMeasureValue'].str.count('<L').sum()
print(count_of_L_values)

In [ ]:
# Replaced '<L' result, which denoted results that were less than the detection limit, with half the value of the minimum result for their respective parameter. Standard practice is to impute half of the minimum detection value but I do not have that information.

# Convert 'ResultMeasureValue' to numeric, ignoring errors to handle '<L'
EPA_df['ResultMeasureValue'] = pd.to_numeric(EPA_df['ResultMeasureValue'], errors='coerce')

# Find the minimum values for each unique 'Parameter'
min_values = EPA_df.groupby('Parameter')['ResultMeasureValue'].min()

# Replace '<L' with half the respective minimum value
for parameter, min_value in min_values.items():
    mask = (EPA_df['Parameter'] == parameter) & (EPA_df['ResultMeasureValue'].isna())
    EPA_df.loc[mask, 'ResultMeasureValue'] = min_value / 2

In [ ]:
# Counted number of NaNs in results.
nan_count = EPA_df['ResultMeasureValue'].isna().sum()
print(nan_count)

The code to replace '<L' with half the minimum value skipped the 8 parameters where all 'ResultMeasureValue' were '<L'. The '<L' was, however, changed to NaN. As this only occured 219 times, occured for only 8 parameters (none of which are the more critical parameters), and I do not have the information to impute values, I will delete these data.

In [ ]:
# Deleted all rows where the result is NaN. Nothing else can be done to fill in these values so the rows are not useful.
EPA_df = EPA_df.dropna(subset=['ResultMeasureValue'])

In [ ]:
# Printed unique data types and their counts to see what's in the results column.

# Counted data types
data_type_counts = EPA_df['ResultMeasureValue'].apply(lambda x: type(x).__name__).value_counts()

# Printed the results
for data_type, count in data_type_counts.items():
    unique_values = EPA_df['ResultMeasureValue'][EPA_df['ResultMeasureValue'].apply(lambda x: type(x).__name__) == data_type].nunique()
    print(f"Data Type: {data_type}, Total Values: {count}, Unique Values: {unique_values}")

In [ ]:
# Counted number of NaNs in results.
nan_count = EPA_df['ResultMeasureValue'].isna().sum()
print(nan_count)

In [ ]:
# Saved to CSV.
EPA_df.to_csv('EPA_data10.csv', index=False)